In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier


print("Cargando datasets...")
df_train = pd.read_csv('/train_IA_PROJECT.csv')
df_statlog = pd.read_csv('/statlog_limpio.csv')

print(f"Filas originales: {len(df_train)}")
print(f"Filas de refuerzo (Statlog): {len(df_statlog)}")


df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes para entrenar: {len(df_total)}")



# Regla fbs: -9 es 0 (Sano)
df_total['fbs'] = df_total['fbs'].replace(['-9.0', '-9', -9, -9.0], 0)

# Convertimos basura a NaN
df_total.replace(['?', '-9.0', '-9', -9], np.nan, inplace=True)

# Asegurar que todo es numérico
for col in df_total.columns:
    if col != 'label':
        df_total[col] = pd.to_numeric(df_total[col], errors='coerce')

# Borrar filas sin label 
df_total.dropna(subset=['label'], inplace=True)



cols_unknown = ['ca', 'thal', 'slope']
for col in cols_unknown:
    df_total[col] = df_total[col].fillna(-1)

# KNN (Para rellenar edad, colesterol, etc.)

cols_restantes = [col for col in df_total.columns if col not in cols_unknown + ['label']]

imputer = KNNImputer(n_neighbors=5)
df_filled = imputer.fit_transform(df_total[cols_restantes])
df_total[cols_restantes] = df_filled


df_total.fillna(0, inplace=True)


X = df_total.drop('label', axis=1)
y = df_total['label'].astype(int)

# Normalización (StandardScaler)
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


smote = SMOTE(k_neighbors=1, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


print("Entrenando XGBoost...")
xgb_model = XGBClassifier(
    n_estimators=400,      
    learning_rate=0.02,     
    max_depth=5,            
    subsample=0.8,          
    colsample_bytree=0.8,  
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_model.fit(X_train_smote, y_train_smote)


y_pred = xgb_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')

print("\n" + "="*30)
print(f"RESULTADOS FINALES (FUSIÓN + XGBOOST):")
print(f"Accuracy: {acc:.4f} ({(acc*100):.2f}%)")
print(f"Micro F1: {f1:.4f}")
print("="*30)


print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

Cargando datasets...
Filas originales: 732
Filas de refuerzo (Statlog): 270
Total pacientes para entrenar: 1002
Aplicando SMOTE (Generando datos sintéticos)...
Entrenando XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:27:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🔥 RESULTADOS FINALES (FUSIÓN + XGBOOST):
Accuracy: 0.6119 (61.19%)
Micro F1: 0.6119

Matriz de Confusión:
[[85  5  3  2  1]
 [ 5 26 14 10  0]
 [ 3  7  6  4  2]
 [ 3  9  3  4  2]
 [ 1  2  0  2  2]]


# Mejora de analisis

In [244]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier


In [ ]:

df_train = pd.read_csv('/train_IA_PROJECT.csv')
df_statlog = pd.read_csv('/statlog_limpio.csv')

print(f"Filas originales (train_IA): {len(df_train)}")
print(f"Filas de refuerzo (Statlog): {len(df_statlog)}")

# Unimos datasets
df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes combinados: {len(df_total)}")


Filas originales (train_IA): 732
Filas de refuerzo (Statlog): 270
Total pacientes combinados: 1002


In [246]:
print("Resumen de tipos:")
print(df_total.dtypes)

print("\nValores especiales por columna:")
for col in df_total.columns:
    vals = df_total[col]
    print("\n=== Columna:", col, "===")
    print("Valores únicos (hasta 10):", vals.unique()[:10])
    print("Número de '?':", (vals == '?').sum())
    print("Número de '-9' (string o num):",
          (vals == '-9').sum() + (vals == '-9.0').sum())


Resumen de tipos:
age         float64
sex         float64
cp          float64
trestbps     object
chol         object
fbs          object
restecg     float64
thalach      object
exang        object
oldpeak      object
slope        object
ca           object
thal         object
label         int64
dtype: object

Valores especiales por columna:

=== Columna: age ===
Valores únicos (hasta 10): [51. 54. 63. 52. 55. 44. 35. 62. 68. 50.]
Número de '?': 0
Número de '-9' (string o num): 0

=== Columna: sex ===
Valores únicos (hasta 10): [1. 0.]
Número de '?': 0
Número de '-9' (string o num): 0

=== Columna: cp ===
Valores únicos (hasta 10): [1. 3. 4. 2.]
Número de '?': 0
Número de '-9' (string o num): 0

=== Columna: trestbps ===
Valores únicos (hasta 10): ['125.0' '120.0' '140' '140.0' '122.0' '135' '100' '130.0' '120' '104.0']
Número de '?': 47
Número de '-9' (string o num): 0

=== Columna: chol ===
Valores únicos (hasta 10): ['213.0' '237.0' '0' '-9.0' '217.0' '169.0' '192.0' '297' '231.0' 

In [247]:
print("\nDistribución de grados de enfermedad (label):")
print(df_total['label'].value_counts().sort_index())
print("\nProporciones:")
print(df_total['label'].value_counts(normalize=True).sort_index())



Distribución de grados de enfermedad (label):
label
0    477
1    276
2    108
3    107
4     34
Name: count, dtype: int64

Proporciones:
label
0    0.476048
1    0.275449
2    0.107784
3    0.106786
4    0.033932
Name: proportion, dtype: float64


In [ ]:

print("\nDistribución de sexo:")
print(df_train['sex'].value_counts(dropna=False))
print(df_statlog['sex'].value_counts(dropna=False))





Distribución de sexo (tal cual viene):
sex
1.0    578
0.0    154
Name: count, dtype: int64
sex
1.0    183
0.0     87
Name: count, dtype: int64


In [ ]:
df = df_total.copy()

# 1) fbs: -9 es 0 
df['fbs'] = df['fbs'].replace(['-9.0', '-9'], 0)

# 2) "?" -> NaN
df = df.replace('?', np.nan)

# 3) Convertir columnas numéricas a float 
num_cols_all = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'label']

for col in num_cols_all:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4) Eliminar filas sin label
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

print(df.dtypes)


age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
label         int64
dtype: object


In [ ]:
# columnas donde -9 significa "prueba no realizada"
cols_not_done = ['chol', 'slope', 'ca', 'thal']

for col in cols_not_done:
    # Flag: se intentó o se pudo hacer la prueba
    df[col + '_not_done'] = (df[col] == -9).astype(int)
    # Reemplazamos -9 por NaN, para imputar luego
    df.loc[df[col] == -9, col] = np.nan


In [251]:
df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
df.loc[df['chol'] == 0, 'chol'] = np.nan


In [252]:
# Continuas "biométricas"
cont_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Categóricas clínicas (enteras 0/1/2/3/4)
cat_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Flags de pruebas no realizadas
not_done_cols = [c + '_not_done' for c in cols_not_done]

target_col = 'label'


In [253]:
print("Porcentaje de NaN por columna:")
print(df[cont_cols + cat_cols].isna().mean().sort_values(ascending=False))


Porcentaje de NaN por columna:
ca          0.478044
thal        0.378244
slope       0.246507
chol        0.155689
fbs         0.057884
oldpeak     0.048902
trestbps    0.047904
exang       0.043912
thalach     0.043912
age         0.000000
restecg     0.000000
sex         0.000000
cp          0.000000
dtype: float64


In [254]:
print("\nDistribución de sexo (0 mujer, 1 hombre):")
print(df['sex'].value_counts(normalize=True))

print("\nDistribución de label por sexo:")
print(pd.crosstab(df['sex'], df['label'], normalize='index'))



Distribución de sexo (0 mujer, 1 hombre):
sex
1.0    0.759481
0.0    0.240519
Name: proportion, dtype: float64

Distribución de label por sexo:
label         0         1         2         3         4
sex                                                    
0.0    0.755187  0.161826  0.037344  0.033195  0.012448
1.0    0.387648  0.311432  0.130092  0.130092  0.040736


In [255]:
from sklearn.impute import KNNImputer

df_imp = df.copy()

knn_imp = KNNImputer(n_neighbors=5)
df_imp[cont_cols] = knn_imp.fit_transform(df_imp[cont_cols])

print("NaN en continuas tras KNN:")
print(df_imp[cont_cols].isna().sum())


NaN en continuas tras KNN:
age         0
trestbps    0
chol        0
thalach     0
oldpeak     0
dtype: int64


In [256]:
for col in cat_cols:
    moda = df_imp[col].mode()[0]
    df_imp[col].fillna(moda, inplace=True)

print("NaN en categóricas tras moda:")
print(df_imp[cat_cols].isna().sum())


NaN en categóricas tras moda:
sex        0
cp         0
fbs        0
restecg    0
exang      0
slope      0
ca         0
thal       0
dtype: int64


/tmp/ipython-input-2969244517.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imp[col].fillna(moda, inplace=True)


In [257]:
features = cont_cols + cat_cols + not_done_cols
X = df_imp[features]
y = df_imp[target_col].astype(int)

print("NaN totales en X:", X.isna().sum().sum())


NaN totales en X: 0


In [258]:
from imblearn.over_sampling import SMOTENC

# Lista completa de columnas de entrada:
feature_cols = features  # = cont_cols + cat_cols + not_done_cols

# Índices de las que son categóricas (incluyendo las *_not_done, que son 0/1)
cat_for_smote = cat_cols + not_done_cols
cat_indices = [feature_cols.index(c) for c in cat_for_smote]

print("Columnas categóricas para SMOTENC:", cat_for_smote)
print("Índices categóricos:", cat_indices)


Columnas categóricas para SMOTENC: ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done']
Índices categóricos: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [259]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Distribución y_train:")
print(y_train.value_counts(normalize=True).sort_index())


Distribución y_train:
label
0    0.475655
1    0.275905
2    0.107366
3    0.107366
4    0.033708
Name: proportion, dtype: float64


In [ ]:
#equilibramos clases

smote_nc = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',  
    k_neighbors=3,
    random_state=42
)

X_train_res, y_train_res = smote_nc.fit_resample(X_train, y_train)

print("Distribución de clases tras SMOTENC:")
print(y_train_res.value_counts().sort_index())


Aplicando SMOTENC para equilibrar clases...
Distribución de clases tras SMOTENC:
label
0    381
1    381
2    381
3    381
4    381
Name: count, dtype: int64


In [261]:
xgb_model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.02,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_model.fit(X_train_res, y_train_res)
y_pred = xgb_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

print("\n" + "="*40)
print("RESULTADOS XGBOOST (SMOTENC + imputación afinada)")
print(f"Accuracy:  {acc:.4f}")
print(f"F1 micro:  {f1_micro:.4f}")
print(f"F1 macro:  {f1_macro:.4f}")
print("="*40)

print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:27:47] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



RESULTADOS XGBOOST (SMOTENC + imputación afinada)
Accuracy:  0.6219
F1 micro:  0.6219
F1 macro:  0.4587

Matriz de confusión:
[[83  6  5  2  0]
 [ 3 27 13 12  0]
 [ 5  5  6  5  1]
 [ 3  8  2  7  1]
 [ 0  2  2  1  2]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.88      0.86      0.87        96
           1       0.56      0.49      0.52        55
           2       0.21      0.27      0.24        22
           3       0.26      0.33      0.29        21
           4       0.50      0.29      0.36         7

    accuracy                           0.62       201
   macro avg       0.48      0.45      0.46       201
weighted avg       0.64      0.62      0.63       201



In [ ]:
# Recuperamos la columna sex de X_test 
sex_test = X_test['sex']

# Máscaras
mask_mujer = (sex_test == 0)
mask_hombre = (sex_test == 1)

print("\nRendimiento en MUJERES:")
print("N mujeres en test:", mask_mujer.sum())
print(classification_report(y_test[mask_mujer], y_pred[mask_mujer], zero_division=0))

print("\nRendimiento en HOMBRES:")
print("N hombres en test:", mask_hombre.sum())
print(classification_report(y_test[mask_hombre], y_pred[mask_hombre], zero_division=0))



Rendimiento en MUJERES:
N mujeres en test: 50
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        37
           1       0.33      0.29      0.31         7
           2       0.33      0.50      0.40         2
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         1

    accuracy                           0.80        50
   macro avg       0.32      0.36      0.33        50
weighted avg       0.74      0.80      0.77        50


Rendimiento en HOMBRES:
N hombres en test: 151
              precision    recall  f1-score   support

           0       0.85      0.78      0.81        59
           1       0.60      0.52      0.56        48
           2       0.20      0.25      0.22        20
           3       0.27      0.39      0.32        18
           4       0.50      0.33      0.40         6

    accuracy                           0.56       151
   macro avg       0.48      0.45   

# Metemos mas mujeres con 3 y 4

In [263]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier

In [ ]:

df_train = pd.read_csv('/train_IA_PROJECT.csv')
df_statlog = pd.read_csv('/statlog_limpio.csv')

print(f"Filas originales (train_IA): {len(df_train)}")
print(f"Filas de refuerzo (Statlog): {len(df_statlog)}")


df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes combinados: {len(df_total)}")

Filas originales (train_IA): 732
Filas de refuerzo (Statlog): 270
Total pacientes combinados: 1002


In [265]:
print("Resumen de tipos:")
print(df_total.dtypes)

print("\nValores especiales por columna:")
for col in df_total.columns:
    vals = df_total[col]
    print("\n=== Columna:", col, "===")
    print("Valores únicos (hasta 10):", vals.unique()[:10])
    print("Número de '?':", (vals == '?').sum())
    print("Número de '-9' (string o num):",
          (vals == '-9').sum() + (vals == '-9.0').sum())

Resumen de tipos:
age         float64
sex         float64
cp          float64
trestbps     object
chol         object
fbs          object
restecg     float64
thalach      object
exang        object
oldpeak      object
slope        object
ca           object
thal         object
label         int64
dtype: object

Valores especiales por columna:

=== Columna: age ===
Valores únicos (hasta 10): [51. 54. 63. 52. 55. 44. 35. 62. 68. 50.]
Número de '?': 0
Número de '-9' (string o num): 0

=== Columna: sex ===
Valores únicos (hasta 10): [1. 0.]
Número de '?': 0
Número de '-9' (string o num): 0

=== Columna: cp ===
Valores únicos (hasta 10): [1. 3. 4. 2.]
Número de '?': 0
Número de '-9' (string o num): 0

=== Columna: trestbps ===
Valores únicos (hasta 10): ['125.0' '120.0' '140' '140.0' '122.0' '135' '100' '130.0' '120' '104.0']
Número de '?': 47
Número de '-9' (string o num): 0

=== Columna: chol ===
Valores únicos (hasta 10): ['213.0' '237.0' '0' '-9.0' '217.0' '169.0' '192.0' '297' '231.0' 

In [266]:
print("\nDistribución de grados de enfermedad (label):")
print(df_total['label'].value_counts().sort_index())
print("\nProporciones:")
print(df_total['label'].value_counts(normalize=True).sort_index())


Distribución de grados de enfermedad (label):
label
0    477
1    276
2    108
3    107
4     34
Name: count, dtype: int64

Proporciones:
label
0    0.476048
1    0.275449
2    0.107784
3    0.106786
4    0.033932
Name: proportion, dtype: float64


In [ ]:

print("\nDistribución de sexo (tal cual viene):")
print(df_train['sex'].value_counts(dropna=False))
print(df_statlog['sex'].value_counts(dropna=False))





Distribución de sexo (tal cual viene):
sex
1.0    578
0.0    154
Name: count, dtype: int64
sex
1.0    183
0.0     87
Name: count, dtype: int64


In [ ]:
df = df_total.copy()


df['fbs'] = df['fbs'].replace(['-9.0', '-9'], 0)


df = df.replace('?', np.nan)

# Convertir columnas numéricas a float 
num_cols_all = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'label']

for col in num_cols_all:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Eliminar filas sin label
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

print(df.dtypes)

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
label         int64
dtype: object


In [ ]:
# columnas donde -9 significa "prueba no realizada"
cols_not_done = ['chol', 'slope', 'ca', 'thal']

for col in cols_not_done:
    # Flag: se intentó o se pudo hacer la prueba
    df[col + '_not_done'] = (df[col] == -9).astype(int)
    # Reemplazamos -9 por NaN, para imputar un valor plausible luego
    df.loc[df[col] == -9, col] = np.nan

In [270]:
df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
df.loc[df['chol'] == 0, 'chol'] = np.nan

In [271]:
# Continuas "biométricas"
cont_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Categóricas clínicas (enteras 0/1/2/3/4)
cat_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Flags de pruebas no realizadas
not_done_cols = [c + '_not_done' for c in cols_not_done]

target_col = 'label'

In [272]:
print("Porcentaje de NaN por columna:")
print(df[cont_cols + cat_cols].isna().mean().sort_values(ascending=False))

Porcentaje de NaN por columna:
ca          0.478044
thal        0.378244
slope       0.246507
chol        0.155689
fbs         0.057884
oldpeak     0.048902
trestbps    0.047904
exang       0.043912
thalach     0.043912
age         0.000000
restecg     0.000000
sex         0.000000
cp          0.000000
dtype: float64


In [273]:
print("\nDistribución de sexo (0 mujer, 1 hombre):")
print(df['sex'].value_counts(normalize=True))

print("\nDistribución de label por sexo:")
print(pd.crosstab(df['sex'], df['label'], normalize='index'))


Distribución de sexo (0 mujer, 1 hombre):
sex
1.0    0.759481
0.0    0.240519
Name: proportion, dtype: float64

Distribución de label por sexo:
label         0         1         2         3         4
sex                                                    
0.0    0.755187  0.161826  0.037344  0.033195  0.012448
1.0    0.387648  0.311432  0.130092  0.130092  0.040736


In [274]:
from sklearn.impute import KNNImputer

df_imp = df.copy()

knn_imp = KNNImputer(n_neighbors=5)
df_imp[cont_cols] = knn_imp.fit_transform(df_imp[cont_cols])

print("NaN en continuas tras KNN:")
print(df_imp[cont_cols].isna().sum())

NaN en continuas tras KNN:
age         0
trestbps    0
chol        0
thalach     0
oldpeak     0
dtype: int64


In [275]:
for col in cat_cols:
    moda = df_imp[col].mode()[0]
    df_imp[col].fillna(moda, inplace=True)

print("NaN en categóricas tras moda:")
print(df_imp[cat_cols].isna().sum())

NaN en categóricas tras moda:
sex        0
cp         0
fbs        0
restecg    0
exang      0
slope      0
ca         0
thal       0
dtype: int64


/tmp/ipython-input-996988591.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imp[col].fillna(moda, inplace=True)


In [276]:
features = cont_cols + cat_cols + not_done_cols
X = df_imp[features]
y = df_imp[target_col].astype(int)

print("NaN totales en X:", X.isna().sum().sum())

NaN totales en X: 0


In [277]:
from imblearn.over_sampling import SMOTENC

# Lista completa de columnas de entrada:
feature_cols = features  # = cont_cols + cat_cols + not_done_cols

# Índices de las que son categóricas (incluyendo las *_not_done, que son 0/1)
cat_for_smote = cat_cols + not_done_cols
cat_indices = [feature_cols.index(c) for c in cat_for_smote]

print("Columnas categóricas para SMOTENC:", cat_for_smote)
print("Índices categóricos:", cat_indices)

Columnas categóricas para SMOTENC: ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done']
Índices categóricos: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [278]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Distribución y_train:")
print(y_train.value_counts(normalize=True).sort_index())

Distribución y_train:
label
0    0.475655
1    0.275905
2    0.107366
3    0.107366
4    0.033708
Name: proportion, dtype: float64


In [ ]:

# X_train, X_test, y_train, y_test

mask_mujer_train = (X_train['sex'] == 0)
mask_grave_train = y_train.isin([3, 4])
mask_mujer_grave_train = mask_mujer_train & mask_grave_train

print("Mujeres en train:", mask_mujer_train.sum())
print("Mujeres con label 3-4 en train:", mask_mujer_grave_train.sum())

print("\nTabla sexo x label en train:")
print(pd.crosstab(X_train['sex'], y_train))


Mujeres en train: 191
Mujeres con label 3-4 en train: 7

Tabla sexo x label en train:
label    0    1   2   3   4
sex                        
0.0    145   32   7   5   2
1.0    236  189  79  81  25


In [280]:
from imblearn.over_sampling import SMOTENC

feature_cols = cont_cols + cat_cols + not_done_cols
X = df_imp[feature_cols]
y = df_imp['label'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

cat_for_smote = cat_cols + not_done_cols
cat_indices = [feature_cols.index(c) for c in cat_for_smote]

smote_nc = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

X_train_res, y_train_res = smote_nc.fit_resample(X_train, y_train)

print("Distribución tras SMOTENC:")
print(y_train_res.value_counts().sort_index())


Distribución tras SMOTENC:
label
0    381
1    381
2    381
3    381
4    381
Name: count, dtype: int64


In [ ]:
#Localizamos mujeres con enfermedad grave en el conjunto resampleado
mask_mujer = (X_train_res['sex'] == 0)
mask_grave = y_train_res.isin([3, 4])
mask_mujer_grave = mask_mujer & mask_grave

print("Mujeres graves en X_train_res:", mask_mujer_grave.sum())

X_mujer_grave = X_train_res[mask_mujer_grave]
y_mujer_grave = y_train_res[mask_mujer_grave]

# Replicamos x numero de veces
factor_oversample = 3

X_extra = pd.concat([X_mujer_grave] * (factor_oversample - 1), ignore_index=True)
y_extra = pd.concat([y_mujer_grave] * (factor_oversample - 1), ignore_index=True)

#Creamos nuevo train con estas filas extra
X_train_final = pd.concat([X_train_res, X_extra], ignore_index=True)
y_train_final = pd.concat([y_train_res, y_extra], ignore_index=True)

print("Tamaño final de train:", len(X_train_final))
print("Distribución sexo x label tras oversampling específico:")
print(pd.crosstab(X_train_final['sex'], y_train_final))


Mujeres graves en X_train_res: 7
Tamaño final de train: 2003
Distribución sexo x label tras oversampling específico:
label    0    1    2    3    4
sex                           
0.0    145   49   12   75   30
1.0    236  332  369  376  379


In [ ]:

sample_weight = np.ones(len(y_train_final), dtype=float)

sex_train_final = X_train_final['sex']
label_train_final = y_train_final

mask_mujer_grave_final = (sex_train_final == 0) & (label_train_final.isin([3, 4]))

# Factor extra para mujeres graves
factor_peso = 3.0   

sample_weight[mask_mujer_grave_final] *= factor_peso

print("Número de mujeres graves con peso extra:", mask_mujer_grave_final.sum())
print("Peso medio global:", sample_weight.mean())


Número de mujeres graves con peso extra: 105
Peso medio global: 1.1048427358961557


In [283]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix

xgb_model_w = XGBClassifier(
    n_estimators=400,
    learning_rate=0.02,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_model_w.fit(X_train_final, y_train_final, sample_weight=sample_weight)

y_pred = xgb_model_w.predict(X_test)

print("\n=== RESULTADOS GLOBALES ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 micro:", f1_score(y_test, y_pred, average='micro'))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:27:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== RESULTADOS GLOBALES ===
Accuracy: 0.6218905472636815
F1 micro: 0.6218905472636815
Matriz de confusión:
 [[83  5  5  3  0]
 [ 2 29 11 13  0]
 [ 5  6  5  4  2]
 [ 2  8  4  6  1]
 [ 0  2  2  1  2]]


In [284]:
sex_test = X_test['sex']

mask_mujer_test = (sex_test == 0)
mask_hombre_test = (sex_test == 1)

print("\nRendimiento en MUJERES (modelo ponderado):")
print("N mujeres en test:", mask_mujer_test.sum())
print(classification_report(y_test[mask_mujer_test], y_pred[mask_mujer_test], zero_division=0))

print("\nRendimiento en HOMBRES (modelo ponderado):")
print("N hombres en test:", mask_hombre_test.sum())
print(classification_report(y_test[mask_hombre_test], y_pred[mask_hombre_test], zero_division=0))



Rendimiento en MUJERES (modelo ponderado):
N mujeres en test: 50
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        37
           1       0.40      0.57      0.47         7
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         1

    accuracy                           0.78        50
   macro avg       0.27      0.30      0.28        50
weighted avg       0.76      0.78      0.77        50


Rendimiento en HOMBRES (modelo ponderado):
N hombres en test: 151
              precision    recall  f1-score   support

           0       0.87      0.81      0.84        59
           1       0.62      0.52      0.57        48
           2       0.19      0.25      0.22        20
           3       0.24      0.33      0.28        18
           4       0.40      0.33      0.36         6

    accuracy                           0.57       15

# Enfoque concatenado

In [ ]:

features = cont_cols + cat_cols + not_done_cols

X_full = df_imp[features]
y_full = df_imp["label"].astype(int)  


In [286]:
from sklearn.model_selection import train_test_split

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_full, y_full,
    test_size=0.2,
    stratify=y_full,
    random_state=42
)


In [287]:
# 0 = sano, 1 = enfermo (cualquier grado 1–4)
y_train_enf = (y_train_all > 0).astype(int)
y_test_enf  = (y_test_all  > 0).astype(int)


In [ ]:
from imblearn.over_sampling import SMOTENC
import numpy as np

feature_cols = features

cat_for_smote = cat_cols + not_done_cols
cat_indices = [feature_cols.index(c) for c in cat_for_smote]

smote_nc_1 = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority', 
    k_neighbors=3,
    random_state=42
)

X1_train_res, y1_train_res = smote_nc_1.fit_resample(X_train_all, y_train_enf)

print("Distribución enfermo vs sano (antes):")
print(y_train_enf.value_counts())
print("\nDistribución enfermo vs sano (después SMOTENC):")
print(y1_train_res.value_counts())


Distribución enfermo vs sano (antes):
label
1    420
0    381
Name: count, dtype: int64

Distribución enfermo vs sano (después SMOTENC):
label
0    420
1    420
Name: count, dtype: int64


In [289]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

xgb_enf = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_enf.fit(X1_train_res, y1_train_res)

# Predicciones en el test
y1_proba_test = xgb_enf.predict_proba(X_test_all)[:, 1]   # prob de estar enfermo
y1_pred_test  = (y1_proba_test >= 0.5).astype(int)

print("=== MODELO 1: ENFERMO vs SANO ===")
print("Accuracy:", accuracy_score(y_test_enf, y1_pred_test))
print("F1 macro:", f1_score(y_test_enf, y1_pred_test, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test_enf, y1_pred_test))
print("\nReporte completo:")
print(classification_report(y_test_enf, y1_pred_test))


=== MODELO 1: ENFERMO vs SANO ===
Accuracy: 0.8805970149253731
F1 macro: 0.8797367371360192
Matriz de confusión:
 [[80 16]
 [ 8 97]]

Reporte completo:
              precision    recall  f1-score   support

           0       0.91      0.83      0.87        96
           1       0.86      0.92      0.89       105

    accuracy                           0.88       201
   macro avg       0.88      0.88      0.88       201
weighted avg       0.88      0.88      0.88       201



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:27:52] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [290]:
sex_test = X_test_all["sex"]

mask_f = (sex_test == 0)  # mujeres
mask_m = (sex_test == 1)  # hombres

print("\n=== MODELO 1 – MUJERES ===")
print("N mujeres en test:", mask_f.sum())
print(classification_report(y_test_enf[mask_f], y1_pred_test[mask_f], zero_division=0))

print("\n=== MODELO 1 – HOMBRES ===")
print("N hombres en test:", mask_m.sum())
print(classification_report(y_test_enf[mask_m], y1_pred_test[mask_m], zero_division=0))



=== MODELO 1 – MUJERES ===
N mujeres en test: 50
              precision    recall  f1-score   support

           0       0.92      0.97      0.95        37
           1       0.91      0.77      0.83        13

    accuracy                           0.92        50
   macro avg       0.92      0.87      0.89        50
weighted avg       0.92      0.92      0.92        50


=== MODELO 1 – HOMBRES ===
N hombres en test: 151
              precision    recall  f1-score   support

           0       0.90      0.75      0.81        59
           1       0.85      0.95      0.90        92

    accuracy                           0.87       151
   macro avg       0.88      0.85      0.86       151
weighted avg       0.87      0.87      0.86       151



In [291]:
# TRAIN: quedarnos solo con enfermos (label 1–4)
mask_train_enfermo = (y_train_all > 0)
X_train_enf = X_train_all[mask_train_enfermo].copy()
y_train_enf_label = y_train_all[mask_train_enfermo].copy()

# TEST: lo mismo
mask_test_enfermo = (y_test_all > 0)
X_test_enf = X_test_all[mask_test_enfermo].copy()
y_test_enf_label = y_test_all[mask_test_enfermo].copy()

print("Enfermos en train:", len(X_train_enf))
print("Enfermos en test:", len(X_test_enf))
print("Distrib train por grado (1–4):")
print(y_train_enf_label.value_counts().sort_index())


Enfermos en train: 420
Enfermos en test: 105
Distrib train por grado (1–4):
label
1    221
2     86
3     86
4     27
Name: count, dtype: int64


In [292]:
# 0 = leve/moderado (1–2), 1 = grave (3–4)
y_train_grave = (y_train_enf_label >= 3).astype(int)
y_test_grave  = (y_test_enf_label  >= 3).astype(int)

print("Distrib train grave vs no grave:")
print(y_train_grave.value_counts())


Distrib train grave vs no grave:
label
0    307
1    113
Name: count, dtype: int64


In [293]:
smote_nc_2 = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',  # sube graves
    k_neighbors=3,
    random_state=42
)

X2_train_res, y2_train_res = smote_nc_2.fit_resample(X_train_enf, y_train_grave)

print("Distribución grave vs no grave tras SMOTENC:")
print(y2_train_res.value_counts())


Distribución grave vs no grave tras SMOTENC:
label
1    307
0    307
Name: count, dtype: int64


In [294]:
xgb_grave = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_grave.fit(X2_train_res, y2_train_res)

y2_proba_test = xgb_grave.predict_proba(X_test_enf)[:,1]
y2_pred_test = (y2_proba_test >= 0.5).astype(int)

print("=== MODELO 2: GRAVE vs NO GRAVE (solo enfermos) ===")
print("Accuracy:", accuracy_score(y_test_grave, y2_pred_test))
print("F1 macro:", f1_score(y_test_grave, y2_pred_test, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test_grave, y2_pred_test))
print("\nReporte completo:")
print(classification_report(y_test_grave, y2_pred_test))


=== MODELO 2: GRAVE vs NO GRAVE (solo enfermos) ===
Accuracy: 0.638095238095238
F1 macro: 0.5474137931034483
Matriz de confusión:
 [[57 20]
 [18 10]]

Reporte completo:
              precision    recall  f1-score   support

           0       0.76      0.74      0.75        77
           1       0.33      0.36      0.34        28

    accuracy                           0.64       105
   macro avg       0.55      0.55      0.55       105
weighted avg       0.65      0.64      0.64       105



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:27:52] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [295]:
sex_test_enf = X_test_enf["sex"]

mask_f_enf = (sex_test_enf == 0)
mask_m_enf = (sex_test_enf == 1)

print("\n=== MODELO 2 – MUJERES ENFERMAS ===")
print("N mujeres enfermas en test:", mask_f_enf.sum())
print(classification_report(y_test_grave[mask_f_enf], y2_pred_test[mask_f_enf], zero_division=0))

print("\n=== MODELO 2 – HOMBRES ENFERMOS ===")
print("N hombres enfermos en test:", mask_m_enf.sum())
print(classification_report(y_test_grave[mask_m_enf], y2_pred_test[mask_m_enf], zero_division=0))



=== MODELO 2 – MUJERES ENFERMAS ===
N mujeres enfermas en test: 13
              precision    recall  f1-score   support

           0       0.60      0.67      0.63         9
           1       0.00      0.00      0.00         4

    accuracy                           0.46        13
   macro avg       0.30      0.33      0.32        13
weighted avg       0.42      0.46      0.44        13


=== MODELO 2 – HOMBRES ENFERMOS ===
N hombres enfermos en test: 92
              precision    recall  f1-score   support

           0       0.78      0.75      0.77        68
           1       0.37      0.42      0.39        24

    accuracy                           0.66        92
   macro avg       0.58      0.58      0.58        92
weighted avg       0.68      0.66      0.67        92



# Redes

In [ ]:


features = cont_cols + cat_cols + not_done_cols
X_full = df_imp[features].copy()
y_full = df_imp["label"].astype(int)   


In [297]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_full_scaled = X_full.copy()
X_full_scaled[cont_cols] = scaler.fit_transform(X_full_scaled[cont_cols])


In [298]:
from sklearn.model_selection import train_test_split

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_full_scaled, y_full,
    test_size=0.2,
    stratify=y_full,
    random_state=42
)


In [299]:
import tensorflow as tf
from tensorflow.keras import models, layers, callbacks
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

n_features = X_train_all.shape[1]
n_classes = len(np.unique(y_train_all))

def build_mlp_5class(input_dim, n_classes):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(n_classes, activation="softmax"))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

model_5 = build_mlp_5class(n_features, n_classes)
model_5.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,781 (14.77 KB)

 Trainable params: 3,589 (14.02 KB)

 Non-trainable params: 192 (768.00 B)

In [300]:
classes = np.unique(y_train_all)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train_all
)
class_weight_dict = {cls: w for cls, w in zip(classes, class_weights)}
class_weight_dict


{np.int64(0): np.float64(0.4204724409448819),
 np.int64(1): np.float64(0.7248868778280543),
 np.int64(2): np.float64(1.8627906976744186),
 np.int64(3): np.float64(1.8627906976744186),
 np.int64(4): np.float64(5.933333333333334)}

In [301]:
es = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=15,
    restore_best_weights=True
)

history_5 = model_5.fit(
    X_train_all, y_train_all,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[es],
    verbose=1
)


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.2210 - loss: 2.3173 - val_accuracy: 0.1304 - val_loss: 1.8296
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2935 - loss: 2.0141 - val_accuracy: 0.1180 - val_loss: 1.8711
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2955 - loss: 1.9037 - val_accuracy: 0.1180 - val_loss: 1.8883
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3211 - loss: 1.7665 - val_accuracy: 0.1304 - val_loss: 1.8602
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3574 - loss: 1.6664 - val_accuracy: 0.1491 - val_loss: 1.8321
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3281 - loss: 1.8292 - val_accuracy: 0.1988 - val_loss: 1.7704
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3901 - loss: 1.5408 - val_accuracy: 0.2484 - val_loss: 1.6900
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3937 - loss: 1.5608 - val_accuracy: 0.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

y_pred_5 = np.argmax(model_5.predict(X_test_all), axis=1)

print("=== RED 5 CLASES – Global ===")
print("Accuracy:", accuracy_score(y_test_all, y_pred_5))
print("F1 macro:", f1_score(y_test_all, y_pred_5, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test_all, y_pred_5))
print("\nReporte:")
print(classification_report(y_test_all, y_pred_5, zero_division=0))

# Por sexo
sex_test = X_test_all["sex"]  

mask_f = (sex_test == 0)
mask_m = (sex_test == 1)

print("\n=== RED 5 CLASES – MUJERES ===")
print("N mujeres:", mask_f.sum())
print(classification_report(y_test_all[mask_f], y_pred_5[mask_f], zero_division=0))

print("\n=== RED 5 CLASES – HOMBRES ===")
print("N hombres:", mask_m.sum())
print(classification_report(y_test_all[mask_m], y_pred_5[mask_m], zero_division=0))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
=== RED 5 CLASES – Global ===
Accuracy: 0.5373134328358209
F1 macro: 0.3583679413794503
Matriz de confusión:
 [[75  9  4  6  2]
 [ 8 21  7 11  8]
 [ 3  6  2  2  9]
 [ 2  4  0  8  7]
 [ 0  1  1  3  2]]

Reporte:
              precision    recall  f1-score   support

           0       0.85      0.78      0.82        96
           1       0.51      0.38      0.44        55
           2       0.14      0.09      0.11        22
           3       0.27      0.38      0.31        21
           4       0.07      0.29      0.11         7

    accuracy                           0.54       201
   macro avg       0.37      0.38      0.36       201
weighted avg       0.59      0.54      0.56       201


=== RED 5 CLASES – MUJERES ===
N mujeres: 50
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        37
           1       0.44      0.57      0.50         7
           2       0.00      0.00      0.00         2

In [303]:
y_train_enf = (y_train_all > 0).astype(int)
y_test_enf  = (y_test_all  > 0).astype(int)


In [304]:
def build_mlp_binary(input_dim):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

model_enf = build_mlp_binary(n_features)
model_enf.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 32)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313 (5.13 KB)

 Trainable params: 1,217 (4.75 KB)

 Non-trainable params: 96 (384.00 B)

In [305]:
classes_enf = np.unique(y_train_enf)
class_weights_enf = compute_class_weight(
    class_weight="balanced",
    classes=classes_enf,
    y=y_train_enf
)
cw_enf = {cls: w for cls, w in zip(classes_enf, class_weights_enf)}
cw_enf


{np.int64(0): np.float64(1.0511811023622046),
 np.int64(1): np.float64(0.9535714285714286)}

In [306]:
es = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=15,
    restore_best_weights=True
)

history_enf = model_enf.fit(
    X_train_all, y_train_enf,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    class_weight=cw_enf,
    callbacks=[es],
    verbose=1
)

proba_enf_test = model_enf.predict(X_test_all).ravel()
y_pred_enf = (proba_enf_test >= 0.5).astype(int)

print("=== NN Modelo 1: ENFERMO vs SANO ===")
print("Accuracy:", accuracy_score(y_test_enf, y_pred_enf))
print("F1 macro:", f1_score(y_test_enf, y_pred_enf, average="macro"))
print(confusion_matrix(y_test_enf, y_pred_enf))
print("\nReporte:")
print(classification_report(y_test_enf, y_pred_enf, zero_division=0))


Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.5702 - loss: 0.8867 - val_accuracy: 0.7329 - val_loss: 0.6219
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5887 - loss: 0.7721 - val_accuracy: 0.7019 - val_loss: 0.5916
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6837 - loss: 0.6129 - val_accuracy: 0.7267 - val_loss: 0.5688
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6672 - loss: 0.6064 - val_accuracy: 0.7329 - val_loss: 0.5485
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7057 - loss: 0.5793 - val_accuracy: 0.7267 - val_loss: 0.5327
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7402 - loss: 0.5250 - val_accuracy: 0.7391 - val_loss: 0.5191
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7892 - loss: 0.4752 - val_accuracy: 0.7516 - val_loss: 0.5057
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7567 - loss: 0.5171 - val_accuracy: 0.7764 - 

In [307]:
print("\n=== NN Modelo 1 – MUJERES ===")
print(classification_report(y_test_enf[mask_f], y_pred_enf[mask_f], zero_division=0))

print("\n=== NN Modelo 1 – HOMBRES ===")
print(classification_report(y_test_enf[mask_m], y_pred_enf[mask_m], zero_division=0))



=== NN Modelo 1 – MUJERES ===
              precision    recall  f1-score   support

           0       0.92      0.97      0.95        37
           1       0.91      0.77      0.83        13

    accuracy                           0.92        50
   macro avg       0.92      0.87      0.89        50
weighted avg       0.92      0.92      0.92        50


=== NN Modelo 1 – HOMBRES ===
              precision    recall  f1-score   support

           0       0.81      0.73      0.77        59
           1       0.84      0.89      0.86        92

    accuracy                           0.83       151
   macro avg       0.82      0.81      0.82       151
weighted avg       0.83      0.83      0.83       151



In [308]:
mask_train_enf = (y_train_all > 0)
mask_test_enf  = (y_test_all  > 0)

X_train_enf2 = X_train_all[mask_train_enf].copy()
y_train_label_enf = y_train_all[mask_train_enf].copy()

X_test_enf2 = X_test_all[mask_test_enf].copy()
y_test_label_enf = y_test_all[mask_test_enf].copy()

print("Pacientes enfermos: train =", len(X_train_enf2), "test =", len(X_test_enf2))
print("Distribución grados (train):")
print(y_train_label_enf.value_counts().sort_index())


Pacientes enfermos: train = 420 test = 105
Distribución grados (train):
label
1    221
2     86
3     86
4     27
Name: count, dtype: int64


In [309]:
y_train_grave = (y_train_label_enf >= 3).astype(int)
y_test_grave  = (y_test_label_enf  >= 3).astype(int)

print("Distrib train grave vs no grave:")
print(y_train_grave.value_counts())


Distrib train grave vs no grave:
label
0    307
1    113
Name: count, dtype: int64


In [310]:
classes_grave = np.unique(y_train_grave)
class_weights_grave = compute_class_weight(
    class_weight="balanced",
    classes=classes_grave,
    y=y_train_grave
)
cw_grave = {cls: w for cls, w in zip(classes_grave, class_weights_grave)}
cw_grave


{np.int64(0): np.float64(0.6840390879478827),
 np.int64(1): np.float64(1.8584070796460177)}

In [311]:
model_grave = build_mlp_binary(n_features)

es2 = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=15,
    restore_best_weights=True
)

history_grave = model_grave.fit(
    X_train_enf2, y_train_grave,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    class_weight=cw_grave,
    callbacks=[es2],
    verbose=1
)

proba_grave_test = model_grave.predict(X_test_enf2).ravel()
y_pred_grave = (proba_grave_test >= 0.5).astype(int)

print("=== NN Modelo 2: GRAVE vs NO GRAVE (solo enfermos) ===")
print("Accuracy:", accuracy_score(y_test_grave, y_pred_grave))
print("F1 macro:", f1_score(y_test_grave, y_pred_grave, average="macro"))
print(confusion_matrix(y_test_grave, y_pred_grave))
print("\nReporte:")
print(classification_report(y_test_grave, y_pred_grave, zero_division=0))


Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.4961 - loss: 0.9197 - val_accuracy: 0.3571 - val_loss: 0.7799
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5462 - loss: 0.7869 - val_accuracy: 0.4167 - val_loss: 0.7525
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5341 - loss: 0.7586 - val_accuracy: 0.4405 - val_loss: 0.7304
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5590 - loss: 0.7337 - val_accuracy: 0.4643 - val_loss: 0.7157
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5630 - loss: 0.7026 - val_accuracy: 0.5476 - val_loss: 0.7047
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5212 - loss: 0.7714 - val_accuracy: 0.5476 - val_loss: 0.7018
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5495 - loss: 0.7357 - val_accuracy: 0.5714 - val_loss: 0.6977
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5545 - loss: 0.7190 - val_accuracy: 0.

# Añadimos cols


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier


df_train = pd.read_csv('/train_IA_PROJECT.csv')
df_statlog = pd.read_csv('/statlog_limpio.csv')

print(f"Filas originales: {len(df_train)}")
print(f"Filas de refuerzo (Statlog): {len(df_statlog)}")

df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes: {len(df_total)}")


Filas originales: 732
Filas de refuerzo (Statlog): 270
Total pacientes: 1002


In [313]:
df = df_total.copy()

# Columnas donde sabemos que -9 significa "prueba no realizada"
cols_not_done_num = ['chol']               # numéricas
cols_not_done_cat = ['slope', 'ca', 'thal']  # categóricas 1–3/0–3/3–7

for col in cols_not_done_num + cols_not_done_cat:
    mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
    df[col + '_not_done'] = mask_nd.astype(int)


In [ ]:
df['fbs_not_done'] = df['fbs'].astype(str).isin(['-9', '-9.0']).astype(int)
df['fbs'] = df['fbs'].replace(['-9', '-9.0'], 0)  


In [315]:
df.replace('?', np.nan, inplace=True)


In [316]:
num_cols_all = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'label']

for col in num_cols_all:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [317]:
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)


In [318]:
cont_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']  # continuas “biométricas”
cat_cols  = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']  # clínicas codificadas

not_done_cols = [c + '_not_done' for c in cols_not_done_num + cols_not_done_cat] + ['fbs_not_done']
target_col = 'label'


In [319]:
df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
df.loc[df['chol'] == 0,      'chol']     = np.nan


In [ ]:
for col in ['slope', 'ca', 'thal']:
    
    df[col] = df[col].fillna(-1)  # -1 = categoría "no disponible / no realizada"


In [321]:
# Número de pruebas críticas NO realizadas
df['n_advanced_not_done'] = df[['chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done']].sum(axis=1)

# Proporción respecto al máximo posible (4 en este caso)
df['prop_advanced_not_done'] = df['n_advanced_not_done'] / 4.0

# Número total de pruebas con -9 en todo el registro (incluyendo fbs)
df['n_total_not_done'] = df[not_done_cols].sum(axis=1)


In [322]:
extra_cols = ['n_advanced_not_done', 'prop_advanced_not_done', 'n_total_not_done']
features = cont_cols + cat_cols + not_done_cols + extra_cols


In [323]:
from sklearn.impute import KNNImputer

df_imp = df.copy()

knn_imp = KNNImputer(n_neighbors=5)
df_imp[cont_cols] = knn_imp.fit_transform(df_imp[cont_cols])

print("NaN en continuas tras KNN:")
print(df_imp[cont_cols].isna().sum())


NaN en continuas tras KNN:
age         0
trestbps    0
chol        0
thalach     0
oldpeak     0
dtype: int64


In [324]:
for col in ['sex', 'cp', 'fbs', 'restecg', 'exang']:
    moda = df_imp[col].mode()[0]
    df_imp[col].fillna(moda, inplace=True)

print("NaN en cat tras moda:")
print(df_imp[cat_cols].isna().sum())


NaN en cat tras moda:
sex        0
cp         0
fbs        0
restecg    0
exang      0
slope      0
ca         0
thal       0
dtype: int64


/tmp/ipython-input-3958482152.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imp[col].fillna(moda, inplace=True)


In [ ]:
X = df_imp[features]
y = df_imp[target_col].astype(int)

feature_cols = features


cat_for_smote = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'] + not_done_cols
cat_indices = [feature_cols.index(c) for c in cat_for_smote]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Distribución original clases y_train:")
print(y_train.value_counts().sort_index())

smote_nc = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

X_train_res, y_train_res = smote_nc.fit_resample(X_train, y_train)

print("\nDistribución clases tras SMOTENC:")
print(y_train_res.value_counts().sort_index())


Distribución original clases y_train:
label
0    381
1    221
2     86
3     86
4     27
Name: count, dtype: int64

Distribución clases tras SMOTENC:
label
0    381
1    381
2    381
3    381
4    381
Name: count, dtype: int64


In [326]:
xgb_model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_model.fit(X_train_res, y_train_res)

y_pred = xgb_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

print("\n" + "="*40)
print("RESULTADOS XGBOOST con features de PRUEBAS NO HECHAS")
print(f"Accuracy:  {acc:.4f}")
print(f"F1 micro:  {f1_micro:.4f}")
print(f"F1 macro:  {f1_macro:.4f}")
print("="*40)

print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte global:")
print(classification_report(y_test, y_pred, zero_division=0))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:28:31] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



RESULTADOS XGBOOST con features de PRUEBAS NO HECHAS
Accuracy:  0.6070
F1 micro:  0.6070
F1 macro:  0.4152

Matriz de confusión:
[[85  4  6  1  0]
 [ 3 26 13 13  0]
 [ 3  6  6  5  2]
 [ 3 11  2  3  2]
 [ 1  2  0  2  2]]

Reporte global:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        96
           1       0.53      0.47      0.50        55
           2       0.22      0.27      0.24        22
           3       0.12      0.14      0.13        21
           4       0.33      0.29      0.31         7

    accuracy                           0.61       201
   macro avg       0.42      0.41      0.42       201
weighted avg       0.62      0.61      0.61       201



# Comparacion de modelos añadiendo columnas


In [327]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

def evaluar_modelo(nombre, modelo, X_train, y_train, X_test, y_test):
    print(f"\n{'='*20} {nombre} {'='*20}")
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average="micro")
    f1_macro = f1_score(y_test, y_pred, average="macro")

    print(f"Accuracy : {acc:.4f}")
    print(f"F1 micro: {f1_micro:.4f}")
    print(f"F1 macro: {f1_macro:.4f}")
    print("\nMatriz de confusión:")
    print(confusion_matrix(y_test, y_pred))
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred, zero_division=0))

    return acc, f1_macro


In [328]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    multi_class="multinomial",
    class_weight="balanced",  # importante por el desbalanceo
    max_iter=30000,
    n_jobs=-1
)

acc_lr, f1_lr = evaluar_modelo(
    "Logistic Regression multinomial",
    log_reg,
    X_train_res, y_train_res,
    X_test, y_test
)



==================== Logistic Regression multinomial ====================


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy : 0.5522
F1 micro: 0.5522
F1 macro: 0.3698

Matriz de confusión:
[[75 10  4  6  1]
 [11 25  9  9  1]
 [ 2  8  2  6  4]
 [ 1  4  2  7  7]
 [ 0  3  1  1  2]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.84      0.78      0.81        96
           1       0.50      0.45      0.48        55
           2       0.11      0.09      0.10        22
           3       0.24      0.33      0.28        21
           4       0.13      0.29      0.18         7

    accuracy                           0.55       201
   macro avg       0.37      0.39      0.37       201
weighted avg       0.58      0.55      0.56       201



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=10000,
    max_depth=None,          
    min_samples_split=5,
    min_samples_leaf=3,
    class_weight="balanced_subsample",  
    random_state=42,
    n_jobs=-1
)

acc_rf, f1_rf = evaluar_modelo(
    "Random Forest",
    rf,
    X_train_res, y_train_res,
    X_test, y_test
)



==================== Random Forest ====================
Accuracy : 0.6169
F1 micro: 0.6169
F1 macro: 0.4400

Matriz de confusión:
[[84  6  2  3  1]
 [ 4 24 14 13  0]
 [ 3  7  5  4  3]
 [ 2  8  0  9  2]
 [ 1  2  0  2  2]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.89      0.88      0.88        96
           1       0.51      0.44      0.47        55
           2       0.24      0.23      0.23        22
           3       0.29      0.43      0.35        21
           4       0.25      0.29      0.27         7

    accuracy                           0.62       201
   macro avg       0.44      0.45      0.44       201
weighted avg       0.63      0.62      0.62       201



In [330]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

acc_gb, f1_gb = evaluar_modelo(
    "Gradient Boosting (sklearn)",
    gb,
    X_train_res, y_train_res,
    X_test, y_test
)



==================== Gradient Boosting (sklearn) ====================
Accuracy : 0.5970
F1 micro: 0.5970
F1 macro: 0.4207

Matriz de confusión:
[[80  9  4  3  0]
 [ 4 29 10 12  0]
 [ 3  7  5  5  2]
 [ 3  9  4  4  1]
 [ 0  2  1  2  2]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.89      0.83      0.86        96
           1       0.52      0.53      0.52        55
           2       0.21      0.23      0.22        22
           3       0.15      0.19      0.17        21
           4       0.40      0.29      0.33         7

    accuracy                           0.60       201
   macro avg       0.43      0.41      0.42       201
weighted avg       0.62      0.60      0.61       201



# ML para mujeres

In [ ]:
df = df_imp.copy()  
features_base = cont_cols + cat_cols + not_done_cols  


In [332]:
# Flags de pruebas anómalas
df["hi_bp"]       = (df["trestbps"] >= 140).astype(int)   # hipertensión
df["hi_chol"]     = (df["chol"]     >= 240).astype(int)   # colesterol alto
df["hi_oldpeak"]  = (df["oldpeak"]  >= 2.0).astype(int)   # ST muy deprimido
df["low_thalach"] = (df["thalach"]  <= 120).astype(int)   # FC máxima baja (mala tolerancia)
df["age_gt_60"]   = (df["age"]      >= 60).astype(int)


In [ ]:
risk_flags = ["hi_bp", "hi_chol", "hi_oldpeak", "low_thalach", "age_gt_60"]
df["n_risk_factors"] = df[risk_flags].sum(axis=1)


advanced_not_done = [c for c in df.columns if c.endswith("_not_done") and c.split("_")[0] in ["chol","slope","ca","thal"]]

df["n_advanced_not_done"]   = df[advanced_not_done].sum(axis=1)
df["prop_advanced_not_done"] = df["n_advanced_not_done"] / len(advanced_not_done)

# Número total de pruebas con -9 / no realizadas
not_done_cols = [c for c in df.columns if c.endswith("_not_done")]
df["n_total_not_done"] = df[not_done_cols].sum(axis=1)


In [334]:
df["age_x_chol"]     = df["age"] * df["chol"]
df["age_x_trestbps"] = df["age"] * df["trestbps"]
df["bp_x_oldpeak"]   = df["trestbps"] * df["oldpeak"]


In [335]:
extra_feats = risk_flags + [
    "n_risk_factors",
    "n_advanced_not_done",
    "prop_advanced_not_done",
    "n_total_not_done",
    "age_x_chol",
    "age_x_trestbps",
    "bp_x_oldpeak"
]

features_fe = features_base + extra_feats

X = df[features_fe]
y = df["label"].astype(int)


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

cat_for_smote = cat_cols + not_done_cols  
cat_indices = [features_fe.index(c) for c in cat_for_smote]

smote_nc = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

X_train_res, y_train_res = smote_nc.fit_resample(X_train, y_train)


In [337]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

xgb_fe = XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_fe.fit(X_train_res, y_train_res)

y_pred = xgb_fe.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 macro:", f1_score(y_test, y_pred, average="macro"))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:30:21] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.6169154228855721
F1 macro: 0.41732719457913336
[[84  6  5  1  0]
 [ 2 26 14 13  0]
 [ 3  8  5  2  4]
 [ 2  9  1  8  1]
 [ 1  3  0  2  1]]
              precision    recall  f1-score   support

           0       0.91      0.88      0.89        96
           1       0.50      0.47      0.49        55
           2       0.20      0.23      0.21        22
           3       0.31      0.38      0.34        21
           4       0.17      0.14      0.15         7

    accuracy                           0.62       201
   macro avg       0.42      0.42      0.42       201
weighted avg       0.63      0.62      0.62       201



In [338]:
import pandas as pd

importancias = pd.Series(xgb_fe.feature_importances_, index=features_fe)
print(importancias.sort_values(ascending=False).head(20))


cp                        0.099878
prop_advanced_not_done    0.071607
sex                       0.054745
slope_not_done            0.048331
thal_not_done             0.043965
ca                        0.042906
oldpeak                   0.039355
n_advanced_not_done       0.038428
fbs                       0.036980
slope                     0.035172
bp_x_oldpeak              0.033740
restecg                   0.033369
hi_oldpeak                0.033334
exang                     0.030704
hi_bp                     0.028844
n_total_not_done          0.028373
thalach                   0.028218
age_x_trestbps            0.028104
n_risk_factors            0.027152
age_x_chol                0.026610
dtype: float32


In [339]:
X = df[features_fe]
y = df["label"].astype(int)

mask_female = (df["sex"] == 0)
X_female = X[mask_female].copy()
y_female = y[mask_female].copy()

print("Distribución de clases en mujeres:")
print(y_female.value_counts().sort_index())


Distribución de clases en mujeres:
label
0    182
1     39
2      9
3      8
4      3
Name: count, dtype: int64


In [ ]:
from imblearn.over_sampling import SMOTENC

cat_for_smote_f = cat_cols + not_done_cols
cat_indices_f = [list(X_female.columns).index(c) for c in cat_for_smote_f]

smote_f = SMOTENC(
    categorical_features=cat_indices_f,
    sampling_strategy='not majority',  
    k_neighbors=2,                     
    random_state=42
)

X_female_res, y_female_res = smote_f.fit_resample(X_female, y_female)

print("Distribución de clases en mujeres tras SMOTENC:")
print(y_female_res.value_counts().sort_index())


Distribución de clases en mujeres tras SMOTENC:
label
0    182
1    182
2    182
3    182
4    182
Name: count, dtype: int64


In [341]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_female_res, y_female_res,
    test_size=0.2,
    stratify=y_female_res,
    random_state=42
)

xgb_female = XGBClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_female.fit(Xf_train, yf_train)

yf_pred = xgb_female.predict(Xf_test)

print("\n=== MODELO ESPECÍFICO MUJERES ===")
print("Accuracy:", accuracy_score(yf_test, yf_pred))
print("F1 macro:", f1_score(yf_test, yf_pred, average="macro"))
print(confusion_matrix(yf_test, yf_pred))
print(classification_report(yf_test, yf_pred, zero_division=0))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:30:24] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== MODELO ESPECÍFICO MUJERES ===
Accuracy: 0.9010989010989011
F1 macro: 0.9000588342317146
[[32  2  0  0  3]
 [ 0 28  1  6  1]
 [ 1  1 33  1  0]
 [ 0  0  0 37  0]
 [ 1  1  0  0 34]]
              precision    recall  f1-score   support

           0       0.94      0.86      0.90        37
           1       0.88      0.78      0.82        36
           2       0.97      0.92      0.94        36
           3       0.84      1.00      0.91        37
           4       0.89      0.94      0.92        36

    accuracy                           0.90       182
   macro avg       0.90      0.90      0.90       182
weighted avg       0.90      0.90      0.90       182



In [342]:
# Test global (X_test, y_test) del modelo general
X_test_global = X_test.copy()
sex_test = X_test_global["sex"]

# Predicciones del modelo global
y_pred_global = xgb_fe.predict(X_test_global)

# Predicciones del modelo específico mujeres (solo para las filas de mujeres)
mask_f_test = (sex_test == 0)

X_test_fem_global = X_test_global[mask_f_test]
y_pred_fem_model = xgb_female.predict(X_test_fem_global)

# Sustituimos en las posiciones de mujeres
y_pred_combined = y_pred_global.copy()
y_pred_combined[mask_f_test] = y_pred_fem_model

print("\n=== MODELO COMBINADO (global + mujeres-específico) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_combined))
print("F1 macro:", f1_score(y_test, y_pred_combined, average="macro"))
print(confusion_matrix(y_test, y_pred_combined))
print(classification_report(y_test, y_pred_combined, zero_division=0))

print("\n=== SOLO MUJERES con modelo combinado ===")
print(classification_report(y_test[mask_f_test], y_pred_combined[mask_f_test], zero_division=0))



=== MODELO COMBINADO (global + mujeres-específico) ===
Accuracy: 0.6417910447761194
F1 macro: 0.4527550951766126
[[83  6  5  1  1]
 [ 1 28 13 13  0]
 [ 3  7  6  2  4]
 [ 1  7  1 11  1]
 [ 1  3  0  2  1]]
              precision    recall  f1-score   support

           0       0.93      0.86      0.90        96
           1       0.55      0.51      0.53        55
           2       0.24      0.27      0.26        22
           3       0.38      0.52      0.44        21
           4       0.14      0.14      0.14         7

    accuracy                           0.64       201
   macro avg       0.45      0.46      0.45       201
weighted avg       0.67      0.64      0.65       201


=== SOLO MUJERES con modelo combinado ===
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        37
           1       0.71      0.71      0.71         7
           2       0.67      1.00      0.80         2
           3       0.75      1.00      0.86   

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier


df = df_imp.copy()  
# Features solo con columnas originales + flags _not_done
features_base = cont_cols + cat_cols + not_done_cols

X = df[features_base].copy()
y = df["label"].astype(int)

print("Shape X:", X.shape)
print("Distribución de clases global:")
print(y.value_counts().sort_index())

# Split global
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


Shape X: (1002, 21)
Distribución de clases global:
label
0    477
1    276
2    108
3    107
4     34
Name: count, dtype: int64


In [ ]:

cat_for_smote = cat_cols + not_done_cols
cat_indices = [features_base.index(c) for c in cat_for_smote]

smote_nc = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

X_train_res, y_train_res = smote_nc.fit_resample(X_train, y_train)

print("Distribución de clases tras SMOTENC (global):")
print(y_train_res.value_counts().sort_index())


xgb_fe = XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_fe.fit(X_train_res, y_train_res)

y_pred_global = xgb_fe.predict(X_test)

print("\n=== MODELO GLOBAL (solo columnas originales + _not_done) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_global))
print("F1 macro:", f1_score(y_test, y_pred_global, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_global))
print("\nReporte global:")
print(classification_report(y_test, y_pred_global, zero_division=0))


Distribución de clases tras SMOTENC (global):
label
0    381
1    381
2    381
3    381
4    381
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:30:25] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== MODELO GLOBAL (solo columnas originales + _not_done) ===
Accuracy: 0.6268656716417911
F1 macro: 0.44386524739556704
Matriz de confusión:
 [[86  4  5  1  0]
 [ 4 26 13 12  0]
 [ 3  6  5  5  3]
 [ 3  8  2  7  1]
 [ 1  3  0  1  2]]

Reporte global:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89        96
           1       0.55      0.47      0.51        55
           2       0.20      0.23      0.21        22
           3       0.27      0.33      0.30        21
           4       0.33      0.29      0.31         7

    accuracy                           0.63       201
   macro avg       0.45      0.44      0.44       201
weighted avg       0.64      0.63      0.63       201



In [345]:
# Subconjunto mujeres
mask_female = (df["sex"] == 0)
X_female = X[mask_female].copy()
y_female = y[mask_female].copy()

print("\nDistribución de clases en mujeres (antes de SMOTE):")
print(y_female.value_counts().sort_index())



Distribución de clases en mujeres (antes de SMOTE):
label
0    182
1     39
2      9
3      8
4      3
Name: count, dtype: int64


In [346]:
from imblearn.over_sampling import SMOTENC

cat_for_smote_f = cat_cols + not_done_cols
cat_indices_f = [list(X_female.columns).index(c) for c in cat_for_smote_f]

smote_f = SMOTENC(
    categorical_features=cat_indices_f,
    sampling_strategy='not majority',  # sube clases minoritarias
    k_neighbors=2,                     # pocas muestras, bajamos k
    random_state=42
)

X_female_res, y_female_res = smote_f.fit_resample(X_female, y_female)

print("\nDistribución de clases en mujeres tras SMOTENC:")
print(y_female_res.value_counts().sort_index())



Distribución de clases en mujeres tras SMOTENC:
label
0    182
1    182
2    182
3    182
4    182
Name: count, dtype: int64


In [347]:
from sklearn.model_selection import train_test_split

Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_female_res, y_female_res,
    test_size=0.2,
    stratify=y_female_res,
    random_state=42
)

xgb_female = XGBClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

xgb_female.fit(Xf_train, yf_train)

yf_pred = xgb_female.predict(Xf_test)

print("\n=== MODELO ESPECÍFICO MUJERES (0–4) ===")
print("Accuracy:", accuracy_score(yf_test, yf_pred))
print("F1 macro:", f1_score(yf_test, yf_pred, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(yf_test, yf_pred))
print("\nReporte mujeres (test mujeres-resampleadas):")
print(classification_report(yf_test, yf_pred, zero_division=0))


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:30:27] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== MODELO ESPECÍFICO MUJERES (0–4) ===
Accuracy: 0.9395604395604396
F1 macro: 0.9388266223882662
Matriz de confusión:
 [[35  1  0  0  1]
 [ 0 30  3  3  0]
 [ 0  2 34  0  0]
 [ 0  0  0 37  0]
 [ 0  1  0  0 35]]

Reporte mujeres (test mujeres-resampleadas):
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        37
           1       0.88      0.83      0.86        36
           2       0.92      0.94      0.93        36
           3       0.93      1.00      0.96        37
           4       0.97      0.97      0.97        36

    accuracy                           0.94       182
   macro avg       0.94      0.94      0.94       182
weighted avg       0.94      0.94      0.94       182



In [348]:
# Copia del test global
X_test_global = X_test.copy()
sex_test = X_test_global["sex"]

# Predicciones del modelo global (ya las tenemos: y_pred_global)
# y_pred_global = xgb_fe.predict(X_test_global)

# Predicciones del modelo específico de mujeres (solo en filas de mujeres)
mask_f_test = (sex_test == 0)

X_test_fem_global = X_test_global[mask_f_test]
y_pred_fem_model = xgb_female.predict(X_test_fem_global)

# Sustituimos en las posiciones de mujeres
y_pred_combined = y_pred_global.copy()
y_pred_combined[mask_f_test] = y_pred_fem_model

print("\n=== MODELO COMBINADO (GLOBAL + MUJERES-ESPECÍFICO) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_combined))
print("F1 macro:", f1_score(y_test, y_pred_combined, average="micro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_combined))
print("\nReporte global combinado:")
print(classification_report(y_test, y_pred_combined, zero_division=0))



=== MODELO COMBINADO (GLOBAL + MUJERES-ESPECÍFICO) ===
Accuracy: 0.6616915422885572
F1 macro: 0.6616915422885572
Matriz de confusión:
 [[87  4  4  1  0]
 [ 3 28 12 12  0]
 [ 3  5  6  5  3]
 [ 2  6  2 10  1]
 [ 1  3  0  1  2]]

Reporte global combinado:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        96
           1       0.61      0.51      0.55        55
           2       0.25      0.27      0.26        22
           3       0.34      0.48      0.40        21
           4       0.33      0.29      0.31         7

    accuracy                           0.66       201
   macro avg       0.49      0.49      0.49       201
weighted avg       0.67      0.66      0.67       201



# Prueba Kaggle

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer


cont_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
cat_cols  = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
not_done_cols = ['chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done', 'fbs_not_done']

features_base = cont_cols + cat_cols + not_done_cols


df_train_raw = df_imp.copy()  

def add_not_done_flags(df):
    # flags -9 para pruebas numéricas/categóricas
    for col in ['chol', 'slope', 'ca', 'thal']:
        mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
        df[col + '_not_done'] = mask_nd.astype(int)
    # fbs
    df['fbs_not_done'] = df['fbs'].astype(str).isin(['-9', '-9.0']).astype(int)
    return df

df_train_raw = add_not_done_flags(df_train_raw)

# convertir '?' a NaN
df_train_raw.replace('?', np.nan, inplace=True)

# convertir a num
for col in cont_cols + cat_cols + ['label']:
    df_train_raw[col] = pd.to_numeric(df_train_raw[col], errors='coerce')

# tratar 0 
df_train_raw.loc[df_train_raw['trestbps'] == 0, 'trestbps'] = np.nan
df_train_raw.loc[df_train_raw['chol'] == 0, 'chol'] = np.nan

# slope/ca/thal: -1 como "no realizada", fbs ya corregido
for col in ['slope', 'ca', 'thal']:
    df_train_raw[col] = df_train_raw[col].fillna(-1)

# imputador continuo sobre TRAIN 
knn_imp = KNNImputer(n_neighbors=5)
df_train_raw[cont_cols] = knn_imp.fit_transform(df_train_raw[cont_cols])

# NaN categóricas -> moda (train)
for col in ['sex', 'cp', 'fbs', 'restecg', 'exang']:
    moda = df_train_raw[col].mode()[0]
    df_train_raw[col].fillna(moda, inplace=True)


df_imp = df_train_raw.copy()


/tmp/ipython-input-2438021352.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_raw[col].fillna(moda, inplace=True)


In [ ]:
from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X = df_imp[features_base]
y = df_imp["label"].astype(int)

# split global
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

#SMOTENC 
cat_for_smote = cat_cols + not_done_cols
cat_indices = [features_base.index(c) for c in cat_for_smote]

smote_nc = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

X_train_res, y_train_res = smote_nc.fit_resample(X_train, y_train)

#XGBoost GLOBAL 
xgb_fe = XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)
xgb_fe.fit(X_train_res, y_train_res)

# --- MODELO MUJERES ---
mask_female = (df_imp["sex"] == 0)
X_female = X[mask_female].copy()
y_female = y[mask_female].copy()

cat_for_smote_f = cat_cols + not_done_cols
cat_indices_f = [list(X_female.columns).index(c) for c in cat_for_smote_f]

smote_f = SMOTENC(
    categorical_features=cat_indices_f,
    sampling_strategy='not majority',
    k_neighbors=2,
    random_state=42
)

X_female_res, y_female_res = smote_f.fit_resample(X_female, y_female)

Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_female_res, y_female_res,
    test_size=0.2,
    stratify=y_female_res,
    random_state=42
)

xgb_female = XGBClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softmax',
    num_class=5,
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)
xgb_female.fit(Xf_train, yf_train)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:18:07] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [18:18:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None, num_class=5, ...)

In [ ]:
import pandas as pd
import numpy as np

# Cargar test de Kaggle 
df_test_raw = pd.read_csv("/test.csv") 

df_test = df_test_raw.copy()

# 1) Flags de pruebas no hechas
df_test = add_not_done_flags(df_test)

# 2) Convertir '?' a NaN
df_test.replace('?', np.nan, inplace=True)

# 3) Convertir a numérico continuas + categóricas
for col in cont_cols + cat_cols:
    df_test[col] = pd.to_numeric(df_test[col], errors='coerce')

# 4) de 0 a NaN en continuas
df_test.loc[df_test['trestbps'] == 0, 'trestbps'] = np.nan
df_test.loc[df_test['chol'] == 0,      'chol']     = np.nan

# 5) slope/ca/thal: -1 como categoría "no realizada" (igual que en train)
for col in ['slope', 'ca', 'thal']:
    df_test[col] = df_test[col].fillna(-1)

# 6) Imputamos continuas con knn_imp entrenado en el train
df_test[cont_cols] = knn_imp.transform(df_test[cont_cols])

# 7) Rellenamos NaN en categóricas con la moda del TRAIN (df_imp)
for col in ['sex', 'cp', 'fbs', 'restecg', 'exang']:
    train_mode = df_imp[col].mode()[0]
    df_test[col].fillna(train_mode, inplace=True)

# 8) Matriz final de features para el modelo
X_test_kaggle = df_test[features_base]



/tmp/ipython-input-1275344275.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test[col].fillna(train_mode, inplace=True)


In [ ]:
# Predicciones del modelo global
y_pred_global_test = xgb_fe.predict(X_test_kaggle)

# Predicciones del modelo específico de mujeres
sex_test_kaggle = X_test_kaggle["sex"]
mask_f_test = (sex_test_kaggle == 0)

X_test_fem = X_test_kaggle[mask_f_test]
y_pred_fem_test = xgb_female.predict(X_test_fem)

# Combinamos para mujeres usamos el modelo específico
y_pred_combined_test = y_pred_global_test.copy()
y_pred_combined_test[mask_f_test] = y_pred_fem_test


In [ ]:
# Cargamos sample_submission
sub = pd.read_csv("/sample_submission.csv")

print("Filas test:", len(X_test_kaggle))
print("Filas sample_submission:", len(sub))

sub["label"] = y_pred_combined_test.astype(int)


sub.to_csv("submission.csv", index=False)
print("✅ submission.csv generado")


Filas test: 184
Filas sample_submission: 184
✅ submission.csv generado


# 2ª Prueba Kaggle



In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier


cont_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

cat_cols  = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']


not_done_cols = ['chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done', 'fbs_not_done']

features_base = cont_cols + cat_cols + not_done_cols
target_col = 'label'



def add_not_done_flags(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()

    
    for col in ['chol', 'slope', 'ca', 'thal', 'fbs']:
        
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df[col + '_not_done'] = mask_nd.astype(int)

    return df


def basic_numeric_clean(df: pd.DataFrame, is_train: bool) -> pd.DataFrame:

    df = df.copy()

    # 1) "?" -> NaN
    df.replace('?', np.nan, inplace=True)

    # 2) -9 para chol/slope/ca/thal -> NaN (prueba no realizada)
    for col in ['chol', 'slope', 'ca', 'thal']:
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df.loc[mask_nd, col] = np.nan

    # 3) fbs: -9 / -9.0 -> 0 
    if 'fbs' in df.columns:
        df['fbs'] = df['fbs'].replace(['-9', '-9.0'], 0)

    # 4) Convertimos columnas numéricas a float/int
    num_cols_all = cont_cols + cat_cols
    if is_train and target_col in df.columns:
        num_cols_all = num_cols_all + [target_col]

    for col in num_cols_all:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 5) 0 raros -> NaN
    if 'trestbps' in df.columns:
        df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
    if 'chol' in df.columns:
        df.loc[df['chol'] == 0, 'chol'] = np.nan

    # 6) En TRAIN: eliminar filas sin label y pasarla a int
    if is_train and target_col in df.columns:
        df = df.dropna(subset=[target_col])
        df[target_col] = df[target_col].astype(int)

    return df




def train_xgb_with_pseudo_labeling(
    X_labeled: pd.DataFrame,
    y_labeled: pd.Series,
    X_unlabeled: pd.DataFrame,
    cat_cols: list,
    not_done_cols: list,
    threshold: float = 0.80,
    k_neighbors_smote: int = 3,
    random_state: int = 42,
):
    """
    Entrena un XGBoost con pseudo-labeling:

    1) Split interno train/valid sobre los datos etiquetados.
    2) SMOTENC en train.
    3) Entrena modelo "profesor" (softprob).
    4) Pseudo-labels sobre X_unlabeled con proba_max >= threshold.
    5) Une train original + pseudo-labels -> nuevo train.
    6) Nuevo split + SMOTENC + XGBoost "alumno".
    7) Devuelve modelo alumno + predicciones de test (X_unlabeled).
    """

    features = list(X_labeled.columns)
    cat_for_smote = cat_cols + not_done_cols
    cat_indices = [features.index(c) for c in cat_for_smote if c in features]

    # 1) Split interno
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_labeled, y_labeled,
        test_size=0.2,
        stratify=y_labeled,
        random_state=random_state
    )

    # 2) SMOTENC en train
    smote_nc = SMOTENC(
        categorical_features=cat_indices,
        sampling_strategy='not majority',
        k_neighbors=k_neighbors_smote,
        random_state=random_state
    )

    X_tr_res, y_tr_res = smote_nc.fit_resample(X_tr, y_tr)

    # 3) MODELO PROFESOR
    xgb_teacher = XGBClassifier(
        n_estimators=400,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.9,
        colsample_bytree=0.9,
        objective='multi:softprob',   # para tener probabilidades
        num_class=5,
        random_state=random_state,
        eval_metric='mlogloss',
        use_label_encoder=False
    )

    xgb_teacher.fit(X_tr_res, y_tr_res)

    # Eval en valid
    proba_val = xgb_teacher.predict_proba(X_val)
    y_val_pred = proba_val.argmax(axis=1)

    print("\n=== PROFESOR (antes de pseudo-labeling) ===")
    print("Accuracy val:", accuracy_score(y_val, y_val_pred))
    print("F1 macro val:", f1_score(y_val, y_val_pred, average="macro"))
    print("Matriz conf val:\n", confusion_matrix(y_val, y_val_pred))
    print(classification_report(y_val, y_val_pred, zero_division=0))

    # 4) Pseudo-labels sobre TEST (X_unlabeled)
    proba_test = xgb_teacher.predict_proba(X_unlabeled)
    max_conf = proba_test.max(axis=1)
    high_conf_mask = max_conf >= threshold

    print(f"\nPseudo-labeling con threshold={threshold}")
    print(f"Muestras test totales: {X_unlabeled.shape[0]}")
    print(f"Muestras test con confianza >= threshold: {high_conf_mask.sum()}")

    X_pseudo = X_unlabeled[high_conf_mask]
    y_pseudo = proba_test[high_conf_mask].argmax(axis=1)

    # 5) Train aumentado
    X_aug = pd.concat([X_labeled, X_pseudo], axis=0)
    y_aug = pd.concat([
        y_labeled,
        pd.Series(y_pseudo, index=X_pseudo.index)
    ])

    print(f"Train original: {X_labeled.shape[0]}")
    print(f"Train aumentado: {X_aug.shape[0]}")

    # 6) Nuevo split + SMOTENC + XGB alumno
    X_tr2, X_val2, y_tr2, y_val2 = train_test_split(
        X_aug, y_aug,
        test_size=0.2,
        stratify=y_aug,
        random_state=random_state
    )

    smote_nc2 = SMOTENC(
        categorical_features=cat_indices,
        sampling_strategy='not majority',
        k_neighbors=k_neighbors_smote,
        random_state=random_state
    )

    X_tr2_res, y_tr2_res = smote_nc2.fit_resample(X_tr2, y_tr2)

    xgb_student = XGBClassifier(
        n_estimators=400,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.9,
        colsample_bytree=0.9,
        objective='multi:softprob',
        num_class=5,
        random_state=random_state,
        eval_metric='mlogloss',
        use_label_encoder=False
    )

    xgb_student.fit(X_tr2_res, y_tr2_res)

    # Eval en valid (después de pseudo-labeling)
    proba_val2 = xgb_student.predict_proba(X_val2)
    y_val2_pred = proba_val2.argmax(axis=1)

    print("\n=== ALUMNO (después de pseudo-labeling) ===")
    print("Accuracy val:", accuracy_score(y_val2, y_val2_pred))
    print("F1 macro val:", f1_score(y_val2, y_val2_pred, average="macro"))
    print("Matriz conf val:\n", confusion_matrix(y_val2, y_val2_pred))
    print(classification_report(y_val2, y_val2_pred, zero_division=0))

    # 7) Predicciones finales sobre TODO el test (X_unlabeled)
    proba_test_final = xgb_student.predict_proba(X_unlabeled)
    y_unlabeled_pred = proba_test_final.argmax(axis=1)

    return xgb_student, y_unlabeled_pred


# ============================================================
# 4. Cargar y limpiar TRAIN (train_IA + statlog)
# ============================================================

print("Cargando datasets de TRAIN...")
df_train = pd.read_csv("/train_IA_PROJECT.csv")
df_statlog = pd.read_csv("/statlog_limpio.csv")

print(f"Filas train_IA_PROJECT: {len(df_train)}")
print(f"Filas statlog_limpio   : {len(df_statlog)}")

# Fusionamos ambos (como en tu notebook)
df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes combinados (train+statlog): {len(df_total)}")

# 1) Flags not_done en TRAIN
df_total = add_not_done_flags(df_total)

# 2) Limpieza básica numérica + label
df_train_clean = basic_numeric_clean(df_total, is_train=True)

# 3) Imputación KNN en continuas (solo con TRAIN)
knn_imp = KNNImputer(n_neighbors=5)
df_train_clean[cont_cols] = knn_imp.fit_transform(df_train_clean[cont_cols])

# 4) NaN en categóricas -> moda (solo TRAIN)
cat_modes = {}
for col in cat_cols:
    moda = df_train_clean[col].mode()[0]
    cat_modes[col] = moda
    df_train_clean[col].fillna(moda, inplace=True)

# Este será tu df_imp definitivo de TRAIN
df_imp = df_train_clean.copy()

print("\nResumen df_imp (TRAIN limpio+imputado):")
print(df_imp[features_base + [target_col]].isna().sum())

# ============================================================
# 5. Cargar y limpiar TEST de Kaggle
# ============================================================

print("\nCargando TEST de Kaggle...")
df_test_raw = pd.read_csv("/test.csv")   # ajusta ruta si hace falta

# 1) Flags not_done en TEST
df_test = add_not_done_flags(df_test_raw)

# 2) Limpieza básica numérica (sin label)
df_test = basic_numeric_clean(df_test, is_train=False)

# 3) Imputación KNN en continuas usando MISMO imputador que TRAIN
df_test[cont_cols] = knn_imp.transform(df_test[cont_cols])

# 4) NaN en categóricas -> usar las modas del TRAIN
for col in cat_cols:
    df_test[col].fillna(cat_modes[col], inplace=True)

print("\nNaNs en TEST después de limpieza:")
print(df_test[features_base].isna().sum())

# ============================================================
# 6. Preparar matrices X e y (global y mujeres)
# ============================================================

X_full = df_imp[features_base].copy()
y_full = df_imp[target_col].astype(int)

X_test_kaggle = df_test[features_base].copy()

print("\nShape X_full (TRAIN):", X_full.shape)
print("Distrib clases TRAIN:")
print(y_full.value_counts().sort_index())

# Subconjunto mujeres en TRAIN y TEST
mask_female_train = (df_imp["sex"] == 0)
mask_female_test  = (df_test["sex"] == 0)

X_female_train = X_full[mask_female_train].copy()
y_female_train = y_full[mask_female_train].copy()

X_female_test = X_test_kaggle[mask_female_test].copy()

print("\nDistrib clases en MUJERES (TRAIN):")
print(y_female_train.value_counts().sort_index())

# ============================================================
# 7. Pseudo-labeling GLOBAL
# ============================================================

print("\n\n================ GLOBAL: PSEUDO-LABELING ================")

model_global, y_pred_test_global = train_xgb_with_pseudo_labeling(
    X_labeled=X_full,
    y_labeled=y_full,
    X_unlabeled=X_test_kaggle,
    cat_cols=cat_cols,
    not_done_cols=not_done_cols,
    threshold=0.80,          # puedes probar 0.85 / 0.90
    k_neighbors_smote=3,
    random_state=42
)

# ============================================================
# 8. Pseudo-labeling ESPECÍFICO MUJERES
# ============================================================

print("\n\n================ MUJERES: PSEUDO-LABELING ================")

model_female, y_pred_test_female = train_xgb_with_pseudo_labeling(
    X_labeled=X_female_train,
    y_labeled=y_female_train,
    X_unlabeled=X_female_test,
    cat_cols=cat_cols,
    not_done_cols=not_done_cols,
    threshold=0.80,
    k_neighbors_smote=2,   # como hacías tú para mujeres
    random_state=42
)

# ============================================================
# 9. Combinar predicciones GLOBAL + MUJERES y crear submission
# ============================================================

# Empezamos del modelo global
y_pred_combined_test = y_pred_test_global.copy()

# Sustituimos en las filas de mujeres por el modelo específico
y_pred_combined_test[mask_female_test.values] = y_pred_test_female

print("\nDistribución predicciones COMBINADAS en test:")
unique, counts = np.unique(y_pred_combined_test, return_counts=True)
print(dict(zip(unique, counts)))

# Cargar sample_submission
sub = pd.read_csv("/sample_submission.csv")  # ajusta ruta si hace falta

print("\nFilas test Kaggle:", len(X_test_kaggle))
print("Filas sample_submission:", len(sub))

# Asignar las predicciones combinadas
sub["label"] = y_pred_combined_test.astype(int)

# Guardar submission
sub.to_csv("submission.csv", index=False)
print("\n✅ submission.csv generado en el directorio actual")


Cargando datasets de TRAIN...
Filas train_IA_PROJECT: 732
Filas statlog_limpio   : 270
Total pacientes combinados (train+statlog): 1002

Resumen df_imp (TRAIN limpio+imputado):
age               0
trestbps          0
chol              0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
label             0
dtype: int64

Cargando TEST de Kaggle...

NaNs en TEST después de limpieza:
age               0
trestbps          0
chol              0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
dtype: 

/tmp/ipython-input-2482918172.py:274: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_clean[col].fillna(moda, inplace=True)
/tmp/ipython-input-2482918172.py:300: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 


=== PROFESOR (antes de pseudo-labeling) ===
Accuracy val: 0.6119402985074627
F1 macro val: 0.44346540695683323
Matriz conf val:
 [[82  6  5  3  0]
 [ 3 28 12 12  0]
 [ 5  6  5  5  1]
 [ 3  9  2  6  1]
 [ 0  2  1  2  2]]
              precision    recall  f1-score   support

           0       0.88      0.85      0.87        96
           1       0.55      0.51      0.53        55
           2       0.20      0.23      0.21        22
           3       0.21      0.29      0.24        21
           4       0.50      0.29      0.36         7

    accuracy                           0.61       201
   macro avg       0.47      0.43      0.44       201
weighted avg       0.63      0.61      0.62       201


Pseudo-labeling con threshold=0.8
Muestras test totales: 184
Muestras test con confianza >= threshold: 79
Train original: 1002
Train aumentado: 1081


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:15:21] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== ALUMNO (después de pseudo-labeling) ===
Accuracy val: 0.6359447004608295
F1 macro val: 0.39060069695944416
Matriz conf val:
 [[98  3  2  4  0]
 [ 9 30 10  8  2]
 [ 2 11  2  6  1]
 [ 1 11  0  7  3]
 [ 1  2  1  2  1]]
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       107
           1       0.53      0.51      0.52        59
           2       0.13      0.09      0.11        22
           3       0.26      0.32      0.29        22
           4       0.14      0.14      0.14         7

    accuracy                           0.64       217
   macro avg       0.39      0.40      0.39       217
weighted avg       0.62      0.64      0.63       217



================ MUJERES: PSEUDO-LABELING ================


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:15:25] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== PROFESOR (antes de pseudo-labeling) ===
Accuracy val: 0.7959183673469388
F1 macro val: 0.2974124809741248
Matriz conf val:
 [[34  2  0  0  1]
 [ 1  5  0  1  1]
 [ 1  1  0  0  0]
 [ 0  2  0  0  0]
 [ 0  0  0  0  0]]
              precision    recall  f1-score   support

           0       0.94      0.92      0.93        37
           1       0.50      0.62      0.56         8
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         0

    accuracy                           0.80        49
   macro avg       0.29      0.31      0.30        49
weighted avg       0.79      0.80      0.79        49


Pseudo-labeling con threshold=0.8
Muestras test totales: 40
Muestras test con confianza >= threshold: 36
Train original: 241
Train aumentado: 277


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:15:26] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== ALUMNO (después de pseudo-labeling) ===
Accuracy val: 0.8214285714285714
F1 macro val: 0.5054112554112554
Matriz conf val:
 [[41  1  0  0  1]
 [ 3  3  1  2  0]
 [ 1  0  1  0  0]
 [ 0  1  0  0  0]
 [ 0  0  0  0  1]]
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        43
           1       0.60      0.33      0.43         9
           2       0.50      0.50      0.50         2
           3       0.00      0.00      0.00         1
           4       0.50      1.00      0.67         1

    accuracy                           0.82        56
   macro avg       0.50      0.56      0.51        56
weighted avg       0.82      0.82      0.81        56


Distribución predicciones COMBINADAS en test:
{np.int64(0): np.int64(84), np.int64(1): np.int64(44), np.int64(2): np.int64(21), np.int64(3): np.int64(24), np.int64(4): np.int64(11)}

Filas test Kaggle: 184
Filas sample_submission: 184

✅ submission.csv generado en el directorio actual


# 3ª PRUEBA KAGGLE

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier

# ============================================================
# 1. Definición de columnas (SIN chol)
# ============================================================

# Continuas biométricas (quitamos chol)  # <<< CAMBIO
cont_cols = ['age', 'trestbps', 'thalach', 'oldpeak']

# Categóricas clínicas
cat_cols  = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Flags de pruebas NO realizadas (mantenemos chol_not_done)  # <<< chol_not_done sigue siendo feature
not_done_cols = ['chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done', 'fbs_not_done']

features_base = cont_cols + cat_cols + not_done_cols
target_col = 'label'

# ============================================================
# 2. Funciones de limpieza compartidas (TRAIN + TEST)
# ============================================================

def add_not_done_flags(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # chol, slope, ca, thal, fbs
    for col in ['chol', 'slope', 'ca', 'thal', 'fbs']:
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df[col + '_not_done'] = mask_nd.astype(int)

    return df


def basic_numeric_clean(df: pd.DataFrame, is_train: bool) -> pd.DataFrame:
    df = df.copy()

    # 1) "?" -> NaN
    df.replace('?', np.nan, inplace=True)

    # 2) -9 para chol/slope/ca/thal -> NaN (prueba no realizada)
    for col in ['chol', 'slope', 'ca', 'thal']:
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df.loc[mask_nd, col] = np.nan

    # 3) fbs: -9 / -9.0 -> 0
    if 'fbs' in df.columns:
        df['fbs'] = df['fbs'].replace(['-9', '-9.0'], 0)

    # 4) Convertir columnas numéricas a float/int
    num_cols_all = cont_cols + cat_cols
    if 'chol' in df.columns and 'chol' not in num_cols_all:
        # por si quieres seguir usando chol en la limpieza
        num_cols_all = num_cols_all + ['chol']
    if is_train and target_col in df.columns:
        num_cols_all = num_cols_all + [target_col]

    for col in num_cols_all:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 5) Zeros imposibles -> NaN
    if 'trestbps' in df.columns:
        df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
    if 'chol' in df.columns:
        df.loc[df['chol'] == 0, 'chol'] = np.nan

    # 6) En TRAIN: eliminar filas sin label y pasarla a int
    if is_train and target_col in df.columns:
        df = df.dropna(subset=[target_col])
        df[target_col] = df[target_col].astype(int)

    return df


# ============================================================
# 3. Pseudo-labeling genérico para XGBoost + SMOTENC
# ============================================================

def train_xgb_with_pseudo_labeling(
    X_labeled: pd.DataFrame,
    y_labeled: pd.Series,
    X_unlabeled: pd.DataFrame,
    cat_cols: list,
    not_done_cols: list,
    threshold: float = 0.80,
    k_neighbors_smote: int = 3,
    random_state: int = 42,
):
    """
    Entrena un XGBoost con pseudo-labeling (profesor + alumno).
    """

    features = list(X_labeled.columns)
    cat_for_smote = cat_cols + not_done_cols
    cat_indices = [features.index(c) for c in cat_for_smote if c in features]

    # 1) Split interno
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_labeled, y_labeled,
        test_size=0.2,
        stratify=y_labeled,
        random_state=random_state
    )

    # 2) SMOTENC en train
    smote_nc = SMOTENC(
        categorical_features=cat_indices,
        sampling_strategy='not majority',
        k_neighbors=k_neighbors_smote,
        random_state=random_state
    )

    X_tr_res, y_tr_res = smote_nc.fit_resample(X_tr, y_tr)

    # 3) MODELO PROFESOR
    xgb_teacher = XGBClassifier(
        n_estimators=400,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.9,
        colsample_bytree=0.9,
        objective='multi:softprob',
        num_class=5,
        random_state=random_state,
        eval_metric='mlogloss',
        use_label_encoder=False
    )

    xgb_teacher.fit(X_tr_res, y_tr_res)

    # Eval en valid
    proba_val = xgb_teacher.predict_proba(X_val)
    y_val_pred = proba_val.argmax(axis=1)

    print("\n=== PROFESOR (antes de pseudo-labeling) ===")
    print("Accuracy val:", accuracy_score(y_val, y_val_pred))
    print("F1 macro val:", f1_score(y_val, y_val_pred, average="macro"))
    print("Matriz conf val:\n", confusion_matrix(y_val, y_val_pred))
    print(classification_report(y_val, y_val_pred, zero_division=0))

    # 4) Pseudo-labels sobre TEST (X_unlabeled)
    proba_test = xgb_teacher.predict_proba(X_unlabeled)
    max_conf = proba_test.max(axis=1)
    high_conf_mask = max_conf >= threshold

    print(f"\nPseudo-labeling con threshold={threshold}")
    print(f"Muestras test totales: {X_unlabeled.shape[0]}")
    print(f"Muestras test con confianza >= threshold: {high_conf_mask.sum()}")

    X_pseudo = X_unlabeled[high_conf_mask]
    y_pseudo = proba_test[high_conf_mask].argmax(axis=1)

    # 5) Train aumentado
    X_aug = pd.concat([X_labeled, X_pseudo], axis=0)
    y_aug = pd.concat([
        y_labeled,
        pd.Series(y_pseudo, index=X_pseudo.index)
    ])

    print(f"Train original: {X_labeled.shape[0]}")
    print(f"Train aumentado: {X_aug.shape[0]}")

    # 6) Nuevo split + SMOTENC + XGB alumno
    X_tr2, X_val2, y_tr2, y_val2 = train_test_split(
        X_aug, y_aug,
        test_size=0.2,
        stratify=y_aug,
        random_state=random_state
    )

    smote_nc2 = SMOTENC(
        categorical_features=cat_indices,
        sampling_strategy='not majority',
        k_neighbors=k_neighbors_smote,
        random_state=random_state
    )

    X_tr2_res, y_tr2_res = smote_nc2.fit_resample(X_tr2, y_tr2)

    xgb_student = XGBClassifier(
        n_estimators=400,
        learning_rate=0.03,
        max_depth=5,
        subsample=0.9,
        colsample_bytree=0.9,
        objective='multi:softprob',
        num_class=5,
        random_state=random_state,
        eval_metric='mlogloss',
        use_label_encoder=False
    )

    xgb_student.fit(X_tr2_res, y_tr2_res)

    # Eval en valid (después de pseudo-labeling)
    proba_val2 = xgb_student.predict_proba(X_val2)
    y_val2_pred = proba_val2.argmax(axis=1)

    print("\n=== ALUMNO (después de pseudo-labeling) ===")
    print("Accuracy val:", accuracy_score(y_val2, y_val2_pred))
    print("F1 macro val:", f1_score(y_val2, y_val2_pred, average="macro"))
    print("Matriz conf val:\n", confusion_matrix(y_val2, y_val2_pred))
    print(classification_report(y_val2, y_val2_pred, zero_division=0))

    # 7) Predicciones finales sobre TODO el test (X_unlabeled)
    proba_test_final = xgb_student.predict_proba(X_unlabeled)
    y_unlabeled_pred = proba_test_final.argmax(axis=1)

    return xgb_student, y_unlabeled_pred


# ============================================================
# 4. Cargar y limpiar TRAIN (train_IA + statlog)
# ============================================================

print("Cargando datasets de TRAIN...")
df_train = pd.read_csv("/train_IA_PROJECT.csv")       # ajusta ruta
df_statlog = pd.read_csv("/statlog_limpio.csv")       # ajusta ruta

print(f"Filas train_IA_PROJECT: {len(df_train)}")
print(f"Filas statlog_limpio   : {len(df_statlog)}")

# Fusionamos ambos
df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes combinados (train+statlog): {len(df_total)}")

# 1) Flags not_done en TRAIN
df_total = add_not_done_flags(df_total)

# 2) Limpieza básica numérica + label
df_train_clean = basic_numeric_clean(df_total, is_train=True)

# 3) Imputación KNN en continuas (solo con TRAIN)
#    OJO: usamos solo cont_cols (sin chol)  # <<< CAMBIO
knn_imp = KNNImputer(n_neighbors=5)
df_train_clean[cont_cols] = knn_imp.fit_transform(df_train_clean[cont_cols])

# 4) NaN en categóricas -> moda (solo TRAIN)
cat_modes = {}
for col in cat_cols:
    moda = df_train_clean[col].mode()[0]
    cat_modes[col] = moda
    df_train_clean[col].fillna(moda, inplace=True)

# 5) Eliminamos chol de TRAIN para que no entre al modelo  # <<< CAMBIO
if 'chol' in df_train_clean.columns:
    df_train_clean = df_train_clean.drop(columns=['chol'])

# df_imp definitivo de TRAIN
df_imp = df_train_clean.copy()

print("\nResumen df_imp (TRAIN limpio+imputado, sin chol en features):")
print(df_imp[features_base + [target_col]].isna().sum())

# ============================================================
# 5. Cargar y limpiar TEST de Kaggle
# ============================================================

print("\nCargando TEST de Kaggle...")
df_test_raw = pd.read_csv("/test.csv")   # ajusta ruta

# 1) Flags not_done en TEST
df_test = add_not_done_flags(df_test_raw)

# 2) Limpieza básica numérica (sin label)
df_test = basic_numeric_clean(df_test, is_train=False)

# 3) Imputación KNN en continuas usando MISMO imputador que TRAIN
df_test[cont_cols] = knn_imp.transform(df_test[cont_cols])

# 4) NaN en categóricas -> usar las modas del TRAIN
for col in cat_cols:
    df_test[col].fillna(cat_modes[col], inplace=True)

# 5) Eliminamos chol también en TEST (por coherencia)  # <<< CAMBIO
if 'chol' in df_test.columns:
    df_test = df_test.drop(columns=['chol'])

print("\nNaNs en TEST después de limpieza (features_base):")
print(df_test[features_base].isna().sum())

# ============================================================
# 6. Preparar matrices X e y (global y mujeres)
# ============================================================

X_full = df_imp[features_base].copy()
y_full = df_imp[target_col].astype(int)

X_test_kaggle = df_test[features_base].copy()

print("\nShape X_full (TRAIN):", X_full.shape)
print("Distrib clases TRAIN:")
print(y_full.value_counts().sort_index())

# Subconjunto mujeres en TRAIN y TEST
mask_female_train = (df_imp["sex"] == 0)
mask_female_test  = (df_test["sex"] == 0)

X_female_train = X_full[mask_female_train].copy()
y_female_train = y_full[mask_female_train].copy()

X_female_test = X_test_kaggle[mask_female_test].copy()

print("\nDistrib clases en MUJERES (TRAIN):")
print(y_female_train.value_counts().sort_index())

# ============================================================
# 7. Pseudo-labeling GLOBAL
# ============================================================

print("\n\n================ GLOBAL: PSEUDO-LABELING (sin chol) ================")

model_global, y_pred_test_global = train_xgb_with_pseudo_labeling(
    X_labeled=X_full,
    y_labeled=y_full,
    X_unlabeled=X_test_kaggle,
    cat_cols=cat_cols,
    not_done_cols=not_done_cols,
    threshold=0.80,
    k_neighbors_smote=3,
    random_state=42
)

# ============================================================
# 8. Pseudo-labeling ESPECÍFICO MUJERES
# ============================================================

print("\n\n================ MUJERES: PSEUDO-LABELING (sin chol) ================")

model_female, y_pred_test_female = train_xgb_with_pseudo_labeling(
    X_labeled=X_female_train,
    y_labeled=y_female_train,
    X_unlabeled=X_female_test,
    cat_cols=cat_cols,
    not_done_cols=not_done_cols,
    threshold=0.80,
    k_neighbors_smote=2,   # como antes para mujeres
    random_state=42
)

# ============================================================
# 9. Combinar predicciones GLOBAL + MUJERES y crear submission
# ============================================================

# Empezamos del modelo global
y_pred_combined_test = y_pred_test_global.copy()

# Sustituimos en las filas de mujeres por el modelo específico
y_pred_combined_test[mask_female_test.values] = y_pred_test_female

print("\nDistribución predicciones COMBINADAS en test (sin chol):")
unique, counts = np.unique(y_pred_combined_test, return_counts=True)
print(dict(zip(unique, counts)))

# Cargar sample_submission
sub = pd.read_csv("/sample_submission.csv")  # ajusta ruta

print("\nFilas test Kaggle:", len(X_test_kaggle))
print("Filas sample_submission:", len(sub))

# Asignar las predicciones combinadas
sub["label"] = y_pred_combined_test.astype(int)

# Guardar submission
sub.to_csv("submission_sin_chol.csv", index=False)
print("\n✅ submission_sin_chol.csv generado en el directorio actual")


Cargando datasets de TRAIN...
Filas train_IA_PROJECT: 732
Filas statlog_limpio   : 270
Total pacientes combinados (train+statlog): 1002

Resumen df_imp (TRAIN limpio+imputado, sin chol en features):
age               0
trestbps          0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
label             0
dtype: int64

Cargando TEST de Kaggle...

NaNs en TEST después de limpieza (features_base):
age               0
trestbps          0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
dtype: in

/tmp/ipython-input-481354112.py:268: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_clean[col].fillna(moda, inplace=True)
/tmp/ipython-input-481354112.py:298: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us


=== PROFESOR (antes de pseudo-labeling) ===
Accuracy val: 0.5671641791044776
F1 macro val: 0.3689081897334331
Matriz conf val:
 [[82  7  4  2  1]
 [ 7 23 11 12  2]
 [ 3  7  5  5  2]
 [ 3 10  3  2  3]
 [ 0  2  1  2  2]]
              precision    recall  f1-score   support

           0       0.86      0.85      0.86        96
           1       0.47      0.42      0.44        55
           2       0.21      0.23      0.22        22
           3       0.09      0.10      0.09        21
           4       0.20      0.29      0.24         7

    accuracy                           0.57       201
   macro avg       0.37      0.38      0.37       201
weighted avg       0.58      0.57      0.57       201


Pseudo-labeling con threshold=0.8
Muestras test totales: 184
Muestras test con confianza >= threshold: 72
Train original: 1002
Train aumentado: 1074


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:16:59] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== ALUMNO (después de pseudo-labeling) ===
Accuracy val: 0.6046511627906976
F1 macro val: 0.3651580087814335
Matriz conf val:
 [[93  6  2  4  1]
 [ 9 26  8 13  2]
 [ 1  9  5  5  2]
 [ 2 11  1  6  2]
 [ 2  4  1  0  0]]
              precision    recall  f1-score   support

           0       0.87      0.88      0.87       106
           1       0.46      0.45      0.46        58
           2       0.29      0.23      0.26        22
           3       0.21      0.27      0.24        22
           4       0.00      0.00      0.00         7

    accuracy                           0.60       215
   macro avg       0.37      0.37      0.37       215
weighted avg       0.61      0.60      0.60       215



================ MUJERES: PSEUDO-LABELING (sin chol) ================


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:17:06] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== PROFESOR (antes de pseudo-labeling) ===
Accuracy val: 0.7346938775510204
F1 macro val: 0.28825396825396826
Matriz conf val:
 [[31  2  0  3  1]
 [ 1  5  0  1  1]
 [ 1  1  0  0  0]
 [ 0  2  0  0  0]
 [ 0  0  0  0  0]]
              precision    recall  f1-score   support

           0       0.94      0.84      0.89        37
           1       0.50      0.62      0.56         8
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         0

    accuracy                           0.73        49
   macro avg       0.29      0.29      0.29        49
weighted avg       0.79      0.73      0.76        49


Pseudo-labeling con threshold=0.8
Muestras test totales: 40
Muestras test con confianza >= threshold: 31
Train original: 241
Train aumentado: 272


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:17:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== ALUMNO (después de pseudo-labeling) ===
Accuracy val: 0.7818181818181819
F1 macro val: 0.2746031746031746
Matriz conf val:
 [[39  2  0  0  1]
 [ 2  4  0  2  0]
 [ 0  2  0  0  0]
 [ 1  1  0  0  0]
 [ 0  1  0  0  0]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        42
           1       0.40      0.50      0.44         8
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1

    accuracy                           0.78        55
   macro avg       0.27      0.29      0.27        55
weighted avg       0.77      0.78      0.77        55


Distribución predicciones COMBINADAS en test (sin chol):
{np.int64(0): np.int64(79), np.int64(1): np.int64(50), np.int64(2): np.int64(23), np.int64(3): np.int64(22), np.int64(4): np.int64(10)}

Filas test Kaggle: 184
Filas sample_submission: 184

✅ submission_sin_chol.csv generado en el di

# 4ª PRUEBA


In [14]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from imblearn.over_sampling import SMOTENC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# ============================================================
# 1. Definición de columnas (SIN chol)
# ============================================================

# Continuas biométricas (quitamos chol)
cont_cols = ['age', 'trestbps', 'thalach', 'oldpeak']

# Categóricas clínicas
cat_cols  = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Flags de pruebas NO realizadas (mantenemos chol_not_done)
not_done_cols = ['chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done', 'fbs_not_done']

features_base = cont_cols + cat_cols + not_done_cols
target_col = 'label'


# ============================================================
# 2. Funciones de limpieza compartidas (TRAIN + TEST)
# ============================================================

def add_not_done_flags(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea columnas *_not_done a partir de -9/-9.0 en ciertas variables.
    No modifica todavía los valores (eso se hace en otra función).
    """
    df = df.copy()

    # chol, slope, ca, thal, fbs
    for col in ['chol', 'slope', 'ca', 'thal', 'fbs']:
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df[col + '_not_done'] = mask_nd.astype(int)

    return df


def basic_numeric_clean(df: pd.DataFrame, is_train: bool) -> pd.DataFrame:
    """
    - "?" -> NaN
    - -9 en chol/slope/ca/thal -> NaN
    - fbs -9 -> 0
    - Convierte columnas a numéricas
    - Marca 0 imposibles como NaN
    - En TRAIN, limpia label
    """
    df = df.copy()

    # 1) "?" -> NaN
    df.replace('?', np.nan, inplace=True)

    # 2) -9 para chol/slope/ca/thal -> NaN (prueba no realizada)
    for col in ['chol', 'slope', 'ca', 'thal']:
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df.loc[mask_nd, col] = np.nan

    # 3) fbs: -9 / -9.0 -> 0 (pero con flag fbs_not_done)
    if 'fbs' in df.columns:
        df['fbs'] = df['fbs'].replace(['-9', '-9.0'], 0)

    # 4) Convertir columnas numéricas a float/int
    num_cols_all = cont_cols + cat_cols
    if 'chol' in df.columns and 'chol' not in num_cols_all:
        num_cols_all = num_cols_all + ['chol']
    if is_train and target_col in df.columns:
        num_cols_all = num_cols_all + [target_col]

    for col in num_cols_all:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 5) Zeros imposibles -> NaN
    if 'trestbps' in df.columns:
        df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
    if 'chol' in df.columns:
        df.loc[df['chol'] == 0, 'chol'] = np.nan

    # 6) En TRAIN: eliminar filas sin label y pasarla a int
    if is_train and target_col in df.columns:
        df = df.dropna(subset=[target_col])
        df[target_col] = df[target_col].astype(int)

    return df


# ============================================================
# 3. Cargar y limpiar TRAIN (train_IA + statlog)
# ============================================================

print("Cargando datasets de TRAIN...")
df_train = pd.read_csv("/train_IA_PROJECT.csv")       # 🔁 ajusta ruta si hace falta
df_statlog = pd.read_csv("/statlog_limpio.csv")       # 🔁 ajusta ruta si hace falta

print(f"Filas train_IA_PROJECT: {len(df_train)}")
print(f"Filas statlog_limpio   : {len(df_statlog)}")

# Fusionamos ambos
df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes combinados (train+statlog): {len(df_total)}")

# 1) Flags not_done en TRAIN
df_total = add_not_done_flags(df_total)

# 2) Limpieza básica numérica + label
df_train_clean = basic_numeric_clean(df_total, is_train=True)

# 3) Imputación KNN en continuas (solo con TRAIN)
knn_imp = KNNImputer(n_neighbors=5)
df_train_clean[cont_cols] = knn_imp.fit_transform(df_train_clean[cont_cols])

# 4) NaN en categóricas -> moda (solo TRAIN)
cat_modes = {}
for col in cat_cols:
    moda = df_train_clean[col].mode()[0]
    cat_modes[col] = moda
    df_train_clean[col].fillna(moda, inplace=True)

# 5) Eliminamos chol de TRAIN para que no entre al modelo
if 'chol' in df_train_clean.columns:
    df_train_clean = df_train_clean.drop(columns=['chol'])

# df_imp definitivo de TRAIN
df_imp = df_train_clean.copy()

print("\nResumen df_imp (TRAIN limpio+imputado, sin chol en features):")
print(df_imp[features_base + [target_col]].isna().sum())


# ============================================================
# 4. Cargar y limpiar TEST de Kaggle
# ============================================================

print("\nCargando TEST de Kaggle...")
df_test_raw = pd.read_csv("/test.csv")   # 🔁 ajusta ruta si hace falta

# 1) Flags not_done en TEST
df_test = add_not_done_flags(df_test_raw)

# 2) Limpieza básica numérica (sin label)
df_test = basic_numeric_clean(df_test, is_train=False)

# 3) Imputación KNN en continuas usando MISMO imputador que TRAIN
df_test[cont_cols] = knn_imp.transform(df_test[cont_cols])

# 4) NaN en categóricas -> usar las modas del TRAIN
for col in cat_cols:
    df_test[col].fillna(cat_modes[col], inplace=True)

# 5) Eliminamos chol también en TEST (por coherencia)
if 'chol' in df_test.columns:
    df_test = df_test.drop(columns=['chol'])

print("\nNaNs en TEST después de limpieza (features_base):")
print(df_test[features_base].isna().sum())


# ============================================================
# 5. Preparar matrices X e y
# ============================================================

X_full = df_imp[features_base].copy()
y_full = df_imp[target_col].astype(int)

X_test_kaggle = df_test[features_base].copy()

print("\nShape X_full (TRAIN):", X_full.shape)
print("Distrib clases TRAIN (0–4):")
print(y_full.value_counts().sort_index())


# ============================================================
# 6. Modelo general: Regresión Logística BINARIA (0 vs 1–4)
# ============================================================

# Etiqueta binaria: 0 = sano, 1 = enfermo
y_bin = (y_full > 0).astype(int)

# Split train/valid
Xb_train, Xb_val, yb_train, yb_val = train_test_split(
    X_full, y_bin,
    test_size=0.2,
    stratify=y_bin,
    random_state=42
)

# SMOTENC para balancear el binario
features = list(X_full.columns)
cat_for_smote = cat_cols + not_done_cols
cat_indices = [features.index(c) for c in cat_for_smote if c in features]

smote_bin = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

Xb_train_res, yb_train_res = smote_bin.fit_resample(Xb_train, yb_train)

# Regresión logística binaria
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    solver='lbfgs'
)

log_reg.fit(Xb_train_res, yb_train_res)

# Métricas en valid binario
yb_val_pred = log_reg.predict(Xb_val)

print("\n========== MODELO GENERAL: LOGISTIC REGRESSION (0 vs 1–4) ==========")
print("Accuracy binario (val):", accuracy_score(yb_val, yb_val_pred))
print("F1 macro binario (val):", f1_score(yb_val, yb_val_pred, average="macro"))
print("Matriz conf binaria (val):\n", confusion_matrix(yb_val, yb_val_pred))
print(classification_report(yb_val, yb_val_pred, zero_division=0))


# ============================================================
# 7. Modelo específico: Random Forest para severidad 1–4
# ============================================================

# Nos quedamos SOLO con enfermos para el modelo de severidad
mask_sick = (y_full > 0)
X_sick = X_full[mask_sick].copy()
y_sick = y_full[mask_sick].copy()   # valores 1–4

# Split train/valid en enfermos
Xs_train, Xs_val, ys_train, ys_val = train_test_split(
    X_sick, y_sick,
    test_size=0.2,
    stratify=y_sick,
    random_state=42
)

# SMOTENC para balancear clases 1–4
smote_sev = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

Xs_train_res, ys_train_res = smote_sev.fit_resample(Xs_train, ys_train)

# Random Forest multiclase
rf_severity = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_severity.fit(Xs_train_res, ys_train_res)

# Métricas en valid severidad
ys_val_pred = rf_severity.predict(Xs_val)

print("\n========== MODELO ESPECÍFICO: RANDOM FOREST (severidad 1–4) ==========")
print("Accuracy severidad (val):", accuracy_score(ys_val, ys_val_pred))
print("F1 macro severidad (val):", f1_score(ys_val, ys_val_pred, average="macro"))
print("Matriz conf severidad (val):\n", confusion_matrix(ys_val, ys_val_pred))
print(classification_report(ys_val, ys_val_pred, zero_division=0))


# ============================================================
# 8. Predicción en TEST: combinación LogReg (0/1) + RF (1–4)
# ============================================================

# 1) Predicciones binarios en test (probabilidad de estar enfermo)
proba_test_bin = log_reg.predict_proba(X_test_kaggle)
p_sick = proba_test_bin[:, 1]

# Puedes tunear este umbral; por defecto 0.5
threshold = 0.5
mask_sick_test = (p_sick >= threshold)

print(f"\nEn test, número de enfermos predichos (p_sick>={threshold}):", mask_sick_test.sum())
print("Número de sanos predichos:", (~mask_sick_test).sum())

# 2) Inicializamos todas las predicciones de test como 0 (sano)
y_test_pred_final = np.zeros(len(X_test_kaggle), dtype=int)

# 3) Para los que el modelo binario dice "enfermo", usamos el RF 1–4
X_test_sick = X_test_kaggle[mask_sick_test]
y_test_sick_pred = rf_severity.predict(X_test_sick)

# Insertamos esas predicciones 1–4 en las posiciones correspondientes
y_test_pred_final[mask_sick_test] = y_test_sick_pred

print("\nDistribución de predicciones finales (0–4) en test:")
unique, counts = np.unique(y_test_pred_final, return_counts=True)
print(dict(zip(unique, counts)))


# ============================================================
# 9. Crear submission
# ============================================================

sub = pd.read_csv("/sample_submission.csv")  # 🔁 ajusta ruta si hace falta

print("\nFilas test Kaggle:", len(X_test_kaggle))
print("Filas sample_submission:", len(sub))

sub["label"] = y_test_pred_final.astype(int)

sub.to_csv("submission_logreg_rf.csv", index=False)
print("\n✅ submission_logreg_rf.csv generado en el directorio actual")


Cargando datasets de TRAIN...
Filas train_IA_PROJECT: 732
Filas statlog_limpio   : 270
Total pacientes combinados (train+statlog): 1002

Resumen df_imp (TRAIN limpio+imputado, sin chol en features):
age               0
trestbps          0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
label             0
dtype: int64

Cargando TEST de Kaggle...

NaNs en TEST después de limpieza (features_base):
age               0
trestbps          0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
dtype: in

/tmp/ipython-input-3527276797.py:133: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_clean[col].fillna(moda, inplace=True)
/tmp/ipython-input-3527276797.py:164: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 


========== MODELO GENERAL: LOGISTIC REGRESSION (0 vs 1–4) ==========
Accuracy binario (val): 0.8407960199004975
F1 macro binario (val): 0.8401272618810897
Matriz conf binaria (val):
 [[78 18]
 [14 91]]
              precision    recall  f1-score   support

           0       0.85      0.81      0.83        96
           1       0.83      0.87      0.85       105

    accuracy                           0.84       201
   macro avg       0.84      0.84      0.84       201
weighted avg       0.84      0.84      0.84       201


========== MODELO ESPECÍFICO: RANDOM FOREST (severidad 1–4) ==========
Accuracy severidad (val): 0.41904761904761906
F1 macro severidad (val): 0.35309716599190283
Matriz conf severidad (val):
 [[29 16  8  2]
 [ 7  7  3  5]
 [ 7  6  5  3]
 [ 2  1  1  3]]
              precision    recall  f1-score   support

           1       0.64      0.53      0.58        55
           2       0.23      0.32      0.27        22
           3       0.29      0.24      0.26        2

# PRUEBA 5ª

In [22]:
# modelo_pseudo_labeling_v2.py

import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

# ============================================================
# 1. Carga de datos
# ============================================================
TRAIN_PATH   = "/train_IA_PROJECT.csv"
EXTRA_PATH   = "/statlog_limpio.csv"   # datos adicionales que ya usabas
TEST_PATH    = "/test.csv"
TARGET_COL   = "label"                # ajusta si se llama distinto

print("=== Cargando datos ===")
df_train   = pd.read_csv(TRAIN_PATH)
df_statlog = pd.read_csv(EXTRA_PATH)
df_test    = pd.read_csv(TEST_PATH)

print("Train shape   :", df_train.shape)
print("Statlog shape :", df_statlog.shape)
print("Test shape    :", df_test.shape)

# ============================================================
# 2. Eliminar 'chol' y añadir features sencillas
# ============================================================
def add_simple_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Convertir a numérico por si vienen como strings
    for col in ["trestbps", "oldpeak", "thalach"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Hipertensión
    if "trestbps" in df.columns:
        df["hi_bp"] = (df["trestbps"] >= 140).astype(int)

    # ST muy deprimido
    if "oldpeak" in df.columns:
        df["hi_oldpeak"] = (df["oldpeak"] >= 2.0).astype(int)

    # Frecuencia cardiaca máxima baja
    if "thalach" in df.columns:
        df["low_thalach"] = (df["thalach"] <= 120).astype(int)

    return df


for df_name, df in [("train", df_train), ("statlog", df_statlog), ("test", df_test)]:
    df_local = df

    # Quitar 'chol' si existe
    if "chol" in df_local.columns:
        df_local = df_local.drop(columns=["chol"])

    # Añadir features derivadas
    df_local = add_simple_features(df_local)

    # Reasignar
    if df_name == "train":
        df_train = df_local
    elif df_name == "statlog":
        df_statlog = df_local
    else:
        df_test = df_local

print("\n=== Después de limpiar y añadir features ===")
print("Train columns:", df_train.columns.tolist())
print("Test  columns:", df_test.columns.tolist())

# ============================================================
# 3. Construir df_train_full (train + statlog) y preparar X, y
# ============================================================
# Nos aseguramos de que df_statlog tenga la columna TARGET_COL
if TARGET_COL not in df_statlog.columns:
    raise ValueError(f"En {EXTRA_PATH} debe existir la columna '{TARGET_COL}' con las etiquetas.")

df_train_full = pd.concat([df_train, df_statlog], axis=0, ignore_index=True)
print("\nTrain_full shape:", df_train_full.shape)

# Columnas a excluir como features
cols_to_drop = [TARGET_COL]
for extra_col in ["id", "ID", "patient", "index", "Unnamed: 0"]:
    if extra_col in df_train_full.columns:
        cols_to_drop.append(extra_col)

cols_to_drop = list(dict.fromkeys(cols_to_drop))  # quitar duplicados

print("Columnas que NO se usarán como features:", cols_to_drop)

# X, y para entrenamiento
y = df_train_full[TARGET_COL].astype(int)
X = df_train_full.drop(columns=cols_to_drop)

# X_test: mismas columnas que X (orden consistente)
X_test = df_test.copy()
# Quitar también posibles columnas tipo id en test
for col in cols_to_drop:
    if col in X_test.columns and col != TARGET_COL:
        X_test = X_test.drop(columns=[col])

# Asegurarnos de tener mismas columnas en train/test (por si acaso)
X_test = X_test[X.columns]

print("\nFeatures finales:", X.columns.tolist())
print("X shape     :", X.shape)
print("X_test shape:", X_test.shape)

# Convertir todo a numérico (por seguridad)
X = X.apply(pd.to_numeric, errors="coerce")
X_test = X_test.apply(pd.to_numeric, errors="coerce")

# ============================================================
# 4. Imputación + escalado
# ============================================================
print("\n=== Imputación y escalado ===")
imputer = SimpleImputer(strategy="median")

X_imp      = imputer.fit_transform(X)
X_test_imp = imputer.transform(X_test)

scaler = StandardScaler()
X_scaled      = scaler.fit_transform(X_imp)
X_test_scaled = scaler.transform(X_test_imp)

print("X_scaled shape     :", X_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)

# ============================================================
# 5. Definición del modelo base (Voting: LR + RF)
# ============================================================
print("\n=== Definiendo modelo base (VotingClassifier: LR + RF) ===")

clf_lr = LogisticRegression(
    solver="lbfgs",
    multi_class="multinomial",
    max_iter=5000,
    C=2.0,               # un pelín menos regularización que el default
    random_state=42,
    n_jobs=-1
)

clf_rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight=None,   # prueba también "balanced_subsample" si quieres
    random_state=42,
    n_jobs=-1
)

model = VotingClassifier(
    estimators=[("lr", clf_lr), ("rf", clf_rf)],
    voting="soft",
    weights=[2, 1]  # dar algo más de peso a LR (suele generalizar mejor)
)

print("\n=== FASE 1: Entrenamiento inicial con datos reales ===")
model.fit(X_scaled, y)
print("Modelo entrenado.")

# ============================================================
# 6. Pseudo-labeling: seleccionar muestras muy seguras (solo clases 0–2)
# ============================================================
print("\n=== FASE 2: Pseudo-labeling ===")
probs = model.predict_proba(X_test_scaled)   # (n_test, n_clases)
preds = model.predict(X_test_scaled)

max_probs = np.max(probs, axis=1)

# Estrategia: top 20% más seguro como candidatos
fraction = 0.2
k = int(len(max_probs) * fraction)

if k > 0:
    sorted_probs = np.sort(max_probs)
    threshold = sorted_probs[-k]  # prob mínima del top 20%
else:
    threshold = 0.95

candidate_indices = np.where(max_probs >= threshold)[0]
print(f"Inicialmente hay {len(candidate_indices)} pseudo-candidatos con prob >= {threshold:.3f} (top {fraction*100:.0f}%).")

# Filtrar SOLO las pseudo-etiquetas que son clases 0, 1 o 2
candidate_preds = preds[candidate_indices]
mask_012 = candidate_preds <= 2
high_conf_indices = candidate_indices[mask_012]

print(f"De ellos, {len(high_conf_indices)} pertenecen a clases 0,1,2 (las que más nos importan).")

if len(high_conf_indices) == 0:
    print("⚠️ No se han encontrado pseudo-labels suficientemente seguros en 0,1,2.")
    print("   Continúo sin pseudo-labeling (modelo inicial).")
    X_augmented = X_scaled
    y_augmented = y.values
    weights = np.ones(len(y_augmented))
else:
    X_pseudo_scaled = X_test_scaled[high_conf_indices]
    y_pseudo = preds[high_conf_indices]

    n_train  = X_scaled.shape[0]
    n_pseudo = X_pseudo_scaled.shape[0]

    print(f"\nUsaremos {n_pseudo} pseudo-ejemplos para reforzar las clases 0,1,2.")

    # Aumentar dataset
    X_augmented = np.vstack([X_scaled, X_pseudo_scaled])
    y_augmented = np.concatenate([y.values, y_pseudo])

    # Pesos: 1.0 para datos reales, 0.3 para pseudo-labels
    weights = np.ones(n_train + n_pseudo)
    weights[n_train:] = 0.3

    print(f"Dataset real : {n_train}")
    print(f"Dataset pseudo: {n_pseudo}")
    print(f"Total        : {n_train + n_pseudo}")

# ============================================================
# 7. Re-entrenamiento final con pesos (datos reales + pseudo)
# ============================================================
print("\n=== FASE 3: Re-entrenamiento final con pseudo-labels (pesan menos) ===")

# Reinstanciamos el modelo para entrenarlo desde cero con el dataset aumentado
model_final = VotingClassifier(
    estimators=[("lr", clf_lr), ("rf", clf_rf)],
    voting="soft",
    weights=[2, 1]
)

model_final.fit(X_augmented, y_augmented, sample_weight=weights)
print("Modelo final entrenado.")

# ============================================================
# 8. Predicción sobre el test y generación de submission
# ============================================================
print("\n=== FASE 4: Predicción final sobre test ===")
y_test_pred = model_final.predict(X_test_scaled)




# ============================================================
# 9. Crear submission con la MISMA ESTRUCTURA que sample_submission
# ============================================================

# Leemos la plantilla exacta
sub = pd.read_csv("/sample_submission.csv")  # usa la ruta real de tu plantilla

print("\nColumnas de sample_submission:")
print(sub.columns)

print("\nFilas test Kaggle:", len(X_test_kaggle))
print("Filas sample_submission:", len(sub))

if len(sub) != len(X_test_kaggle):
    raise ValueError(
        f"❌ El número de filas del test ({len(X_test_kaggle)}) "
        f"no coincide con sample_submission ({len(sub)})"
    )

# Detectamos la(s) columna(s) de predicción.
# Caso típico: hay una columna 'label', pero si no, usamos la última columna.
if "label" in sub.columns:
    label_col = "label"
else:
    # Tomamos la última columna como columna de predicción
    label_col = sub.columns[-1]
    print(f"⚠️ No se encontró columna 'label'. "
          f"Usando '{label_col}' como columna de predicción.")

# Asignamos nuestras predicciones finales (0–4) a esa columna
sub[label_col] = y_test_pred_final.astype(int)

# Guardamos el CSV con el mismo formato de la plantilla
sub.to_csv("submission_logreg_rf.csv", index=False)
print("\n✅ submission_logreg_rf.csv generado con la misma estructura que sample_submission.csv")


=== Cargando datos ===
Train shape   : (732, 14)
Statlog shape : (270, 14)
Test shape    : (184, 13)

=== Después de limpiar y añadir features ===
Train columns: ['age', 'sex', 'cp', 'trestbps', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'label', 'hi_bp', 'hi_oldpeak', 'low_thalach']
Test  columns: ['age', 'sex', 'cp', 'trestbps', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hi_bp', 'hi_oldpeak', 'low_thalach']

Train_full shape: (1002, 16)
Columnas que NO se usarán como features: ['label']

Features finales: ['age', 'sex', 'cp', 'trestbps', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hi_bp', 'hi_oldpeak', 'low_thalach']
X shape     : (1002, 15)
X_test shape: (184, 15)

=== Imputación y escalado ===
X_scaled shape     : (1002, 15)
X_test_scaled shape: (184, 15)

=== Definiendo modelo base (VotingClassifier: LR + RF) ===

=== FASE 1: Entrenamiento inicial con datos reales ===


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Modelo entrenado.

=== FASE 2: Pseudo-labeling ===
Inicialmente hay 36 pseudo-candidatos con prob >= 0.863 (top 20%).
De ellos, 36 pertenecen a clases 0,1,2 (las que más nos importan).

Usaremos 36 pseudo-ejemplos para reforzar las clases 0,1,2.
Dataset real : 1002
Dataset pseudo: 36
Total        : 1038

=== FASE 3: Re-entrenamiento final con pseudo-labels (pesan menos) ===


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Modelo final entrenado.

=== FASE 4: Predicción final sobre test ===

Columnas de sample_submission:
Index(['ID', 'label'], dtype='object')

Filas test Kaggle: 184
Filas sample_submission: 184

✅ submission_logreg_rf.csv generado con la misma estructura que sample_submission.csv


In [23]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from imblearn.over_sampling import SMOTENC
from xgboost import XGBClassifier

# ============================================================
# 1. Definición de columnas (CON chol)
# ============================================================

# Continuas biométricas
cont_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Categóricas clínicas
cat_cols  = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Flags de pruebas NO realizadas
not_done_cols = ['chol_not_done', 'slope_not_done', 'ca_not_done', 'thal_not_done', 'fbs_not_done']

features_base = cont_cols + cat_cols + not_done_cols
target_col = 'label'


# ============================================================
# 2. Funciones de limpieza compartidas (TRAIN + TEST)
# ============================================================

def add_not_done_flags(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea columnas *_not_done a partir de -9/-9.0 en chol/slope/ca/thal/fbs.
    """
    df = df.copy()

    for col in ['chol', 'slope', 'ca', 'thal', 'fbs']:
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df[col + '_not_done'] = mask_nd.astype(int)

    return df


def basic_numeric_clean(df: pd.DataFrame, is_train: bool) -> pd.DataFrame:
    """
    - "?" -> NaN
    - -9 en chol/slope/ca/thal -> NaN
    - fbs -9 -> 0
    - Convierte columnas a numéricas
    - Marca 0 imposibles como NaN
    - En TRAIN, limpia label
    """
    df = df.copy()

    # 1) "?" -> NaN
    df.replace('?', np.nan, inplace=True)

    # 2) -9 para chol/slope/ca/thal -> NaN (prueba no realizada)
    for col in ['chol', 'slope', 'ca', 'thal']:
        if col in df.columns:
            mask_nd = df[col].astype(str).isin(['-9', '-9.0'])
            df.loc[mask_nd, col] = np.nan

    # 3) fbs: -9 / -9.0 -> 0 (pero con flag fbs_not_done)
    if 'fbs' in df.columns:
        df['fbs'] = df['fbs'].replace(['-9', '-9.0'], 0)

    # 4) Convertir columnas numéricas a float/int
    num_cols_all = cont_cols + cat_cols
    if is_train and target_col in df.columns:
        num_cols_all = num_cols_all + [target_col]

    for col in num_cols_all:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 5) Zeros imposibles -> NaN
    if 'trestbps' in df.columns:
        df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
    if 'chol' in df.columns:
        df.loc[df['chol'] == 0, 'chol'] = np.nan

    # 6) En TRAIN: eliminar filas sin label y pasarla a int
    if is_train and target_col in df.columns:
        df = df.dropna(subset=[target_col])
        df[target_col] = df[target_col].astype(int)

    return df


# ============================================================
# 3. Cargar y limpiar TRAIN (train_IA + statlog)
# ============================================================

print("Cargando datasets de TRAIN...")
df_train = pd.read_csv("/train_IA_PROJECT.csv")       # ajusta ruta
df_statlog = pd.read_csv("/statlog_limpio.csv")       # ajusta ruta

print(f"Filas train_IA_PROJECT: {len(df_train)}")
print(f"Filas statlog_limpio   : {len(df_statlog)}")

# Fusionamos ambos
df_total = pd.concat([df_train, df_statlog], ignore_index=True)
print(f"Total pacientes combinados (train+statlog): {len(df_total)}")

# 1) Flags not_done en TRAIN
df_total = add_not_done_flags(df_total)

# 2) Limpieza básica numérica + label
df_train_clean = basic_numeric_clean(df_total, is_train=True)

# 3) Imputación KNN en continuas (solo con TRAIN)
knn_imp = KNNImputer(n_neighbors=5)
df_train_clean[cont_cols] = knn_imp.fit_transform(df_train_clean[cont_cols])

# 4) NaN en categóricas -> moda (solo TRAIN)
cat_modes = {}
for col in cat_cols:
    moda = df_train_clean[col].mode()[0]
    cat_modes[col] = moda
    df_train_clean[col].fillna(moda, inplace=True)

# df_imp definitivo de TRAIN
df_imp = df_train_clean.copy()

print("\nResumen df_imp (TRAIN limpio+imputado):")
print(df_imp[features_base + [target_col]].isna().sum())


# ============================================================
# 4. Cargar y limpiar TEST de Kaggle
# ============================================================

print("\nCargando TEST de Kaggle...")
df_test_raw = pd.read_csv("/test.csv")   # ajusta ruta

# 1) Flags not_done en TEST
df_test = add_not_done_flags(df_test_raw)

# 2) Limpieza básica numérica (sin label)
df_test = basic_numeric_clean(df_test, is_train=False)

# 3) Imputación KNN en continuas usando MISMO imputador que TRAIN
df_test[cont_cols] = knn_imp.transform(df_test[cont_cols])

# 4) NaN en categóricas -> usar las modas del TRAIN
for col in cat_cols:
    df_test[col].fillna(cat_modes[col], inplace=True)

print("\nNaNs en TEST después de limpieza (features_base):")
print(df_test[features_base].isna().sum())


# ============================================================
# 5. Preparar matrices X e y
# ============================================================

X_full = df_imp[features_base].copy()
y_full = df_imp[target_col].astype(int)

X_test_kaggle = df_test[features_base].copy()

print("\nShape X_full (TRAIN):", X_full.shape)
print("Distrib clases TRAIN (0–4):")
print(y_full.value_counts().sort_index())


# ============================================================
# 6. Modelo general: XGB BINARIO (0 vs 1–4)
# ============================================================

# Etiqueta binaria: 0 = sano, 1 = enfermo
y_bin = (y_full > 0).astype(int)

# Split train/valid
Xb_train, Xb_val, yb_train, yb_val = train_test_split(
    X_full, y_bin,
    test_size=0.2,
    stratify=y_bin,
    random_state=42
)

# SMOTENC para balancear el binario
features = list(X_full.columns)
cat_for_smote = cat_cols + not_done_cols
cat_indices = [features.index(c) for c in cat_for_smote if c in features]

from imblearn.over_sampling import SMOTENC
smote_bin = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

Xb_train_res, yb_train_res = smote_bin.fit_resample(Xb_train, yb_train)

xgb_bin = XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='binary:logistic',
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False,
)

xgb_bin.fit(Xb_train_res, yb_train_res)

yb_val_proba = xgb_bin.predict_proba(Xb_val)[:, 1]
yb_val_pred = (yb_val_proba >= 0.5).astype(int)

print("\n========== MODELO GENERAL: XGB BINARIO (0 vs 1–4) ==========")
print("Accuracy binario (val):", accuracy_score(yb_val, yb_val_pred))
print("F1 macro binario (val):", f1_score(yb_val, yb_val_pred, average="macro"))
print("Matriz conf binaria (val):\n", confusion_matrix(yb_val, yb_val_pred))
print(classification_report(yb_val, yb_val_pred, zero_division=0))


# ============================================================
# 7. Modelo específico: XGB multiclase para severidad 1–4
# ============================================================

# Nos quedamos SOLO con enfermos para el modelo de severidad
mask_sick = (y_full > 0)
X_sick = X_full[mask_sick].copy()
y_sick = y_full[mask_sick].copy()   # valores 1–4

# Vamos a relabelar a 0–3 internamente para XGBoost (opcional pero cómodo)
y_sick_04 = y_sick - 1  # 1–4 -> 0–3

# Split train/valid en enfermos
Xs_train, Xs_val, ys_train, ys_val = train_test_split(
    X_sick, y_sick_04,
    test_size=0.2,
    stratify=y_sick_04,
    random_state=42
)

# SMOTENC para balancear clases 0–3 (equivalentes a 1–4)
smote_sev = SMOTENC(
    categorical_features=cat_indices,
    sampling_strategy='not majority',
    k_neighbors=3,
    random_state=42
)

Xs_train_res, ys_train_res = smote_sev.fit_resample(Xs_train, ys_train)

xgb_severity = XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='multi:softprob',
    num_class=4,          # clases 0,1,2,3 (que luego mapeamos a 1–4)
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False,
)

xgb_severity.fit(Xs_train_res, ys_train_res)

ys_val_proba = xgb_severity.predict_proba(Xs_val)
ys_val_pred_04 = ys_val_proba.argmax(axis=1)
ys_val_pred = ys_val_pred_04 + 1  # volver a 1–4

print("\n========== MODELO ESPECÍFICO: XGB SEVERIDAD (1–4) ==========")
print("Accuracy severidad (val):", accuracy_score(ys_val + 1, ys_val_pred))
print("F1 macro severidad (val):", f1_score(ys_val + 1, ys_val_pred, average="macro"))
print("Matriz conf severidad (val):\n", confusion_matrix(ys_val + 1, ys_val_pred))
print(classification_report(ys_val + 1, ys_val_pred, zero_division=0))


# ============================================================
# 8. Predicción en TEST: combinación jerárquica
# ============================================================

# 1) Probabilidad de estar enfermo en test
proba_test_bin = xgb_bin.predict_proba(X_test_kaggle)
p_sano = proba_test_bin[:, 0]
p_enfermo = proba_test_bin[:, 1]

# 2) Probabilidades de severidad condicional (1–4) en test
proba_sev_cond_04 = xgb_severity.predict_proba(X_test_kaggle)  # shape (n,4)
# Convertimos a 1–4 solo para claridad si queremos luego, pero ahora usamos 0–3

# 3) Combinamos:
#    P(0) = P(sano)
#    P(k) = P(enfermo) * P(severidad=k | enfermo), k=1..4
n_test = X_test_kaggle.shape[0]
proba_final = np.zeros((n_test, 5))

# P(clase 0)
proba_final[:, 0] = p_sano

# P(clases 1–4)
for k in range(4):  # k=0..3 -> clase real = k+1
    proba_final[:, k+1] = p_enfermo * proba_sev_cond_04[:, k]

# 4) Predicción final = argmax sobre 0–4
y_test_pred_final = proba_final.argmax(axis=1).astype(int)

print("\nDistribución de predicciones finales (0–4) en test:")
unique, counts = np.unique(y_test_pred_final, return_counts=True)
print(dict(zip(unique, counts)))


# ============================================================
# 9. Crear submission con la MISMA ESTRUCTURA que sample_submission
# ============================================================

sub = pd.read_csv("/sample_submission.csv")  # usa tu plantilla real

print("\nColumnas de sample_submission:")
print(sub.columns)

print("\nFilas test Kaggle:", len(X_test_kaggle))
print("Filas sample_submission:", len(sub))

if len(sub) != len(X_test_kaggle):
    raise ValueError(
        f"❌ El número de filas del test ({len(X_test_kaggle)}) "
        f"no coincide con sample_submission ({len(sub)})"
    )

# Detectamos la columna de predicción
if "label" in sub.columns:
    label_col = "label"
else:
    label_col = sub.columns[-1]
    print(f"⚠️ No se encontró columna 'label'. "
          f"Usando '{label_col}' como columna de predicción.")

sub[label_col] = y_test_pred_final

sub.to_csv("submission_xgb_bin_xgb_sev.csv", index=False)
print("\n✅ submission_xgb_bin_xgb_sev.csv generado con la misma estructura que sample_submission.csv")


Cargando datasets de TRAIN...
Filas train_IA_PROJECT: 732
Filas statlog_limpio   : 270
Total pacientes combinados (train+statlog): 1002


/tmp/ipython-input-2935985748.py:128: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_clean[col].fillna(moda, inplace=True)
/tmp/ipython-input-2935985748.py:155: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 


Resumen df_imp (TRAIN limpio+imputado):
age               0
trestbps          0
chol              0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
label             0
dtype: int64

Cargando TEST de Kaggle...

NaNs en TEST después de limpieza (features_base):
age               0
trestbps          0
chol              0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
slope_not_done    0
ca_not_done       0
thal_not_done     0
fbs_not_done      0
dtype: int64

Shape X_full (TRAIN): (1002, 18)
Distrib clases TRAIN (0–4):
label
0    477
1    276
2    108
3    107
4     34
N

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:17:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



========== MODELO GENERAL: XGB BINARIO (0 vs 1–4) ==========
Accuracy binario (val): 0.8855721393034826
F1 macro binario (val): 0.8850143017037682
Matriz conf binaria (val):
 [[82 14]
 [ 9 96]]
              precision    recall  f1-score   support

           0       0.90      0.85      0.88        96
           1       0.87      0.91      0.89       105

    accuracy                           0.89       201
   macro avg       0.89      0.88      0.89       201
weighted avg       0.89      0.89      0.89       201



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:17:11] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



========== MODELO ESPECÍFICO: XGB SEVERIDAD (1–4) ==========
Accuracy severidad (val): 0.44761904761904764
F1 macro severidad (val): 0.3749299929992999
Matriz conf severidad (val):
 [[31 12  9  3]
 [ 6  5  5  6]
 [ 8  4  8  1]
 [ 1  2  1  3]]
              precision    recall  f1-score   support

           1       0.67      0.56      0.61        55
           2       0.22      0.23      0.22        22
           3       0.35      0.38      0.36        21
           4       0.23      0.43      0.30         7

    accuracy                           0.45       105
   macro avg       0.37      0.40      0.37       105
weighted avg       0.48      0.45      0.46       105


Distribución de predicciones finales (0–4) en test:
{np.int64(0): np.int64(87), np.int64(1): np.int64(45), np.int64(2): np.int64(21), np.int64(3): np.int64(25), np.int64(4): np.int64(6)}

Columnas de sample_submission:
Index(['ID', 'label'], dtype='object')

Filas test Kaggle: 184
Filas sample_submission: 184

✅ submis